In [ ]:
import xarray as xr
import numpy  as np
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.style import use as usestyle
from matplotlib import colors
from matplotlib.colors import BoundaryNorm
import matplotlib as mpl
from time import perf_counter
import xesmf
import cartopy.crs as ccrs
from cartopy import feature as cfeature
# import metpy as mpy
import os
from pandas import date_range
from glob import glob
from pandas import date_range
from datetime import datetime, timedelta
from matplotlib.lines import Line2D
from matplotlib import ticker as mticker
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
from astropy.convolution import convolve

In [ ]:
# usestyle("paperplots.mplstyle")
from matplotlib import font_manager as fm
fontdir = "/home/ascheb/libfonts/*.ttf"
for fpath in glob(fontdir):
    print(fpath)
    fm.fontManager.addfont(fpath)

usestyle("paperplots.mplstyle")
from matplotlib import rcParams
rcParams["figure.titlesize"] = 12
rcParams["axes.titlesize"] = 10
rcParams["axes.labelsize"] = 8
rcParams["xtick.labelsize"] = 8
rcParams["ytick.labelsize"] = 8
rcParams["axes.linewidth"] = 0.4

In [ ]:
class MidpointNormalize(colors.Normalize):
    def __init__(self, vmin=None, vmax=None, vcenter=None, clip=False):
        self.vcenter = vcenter
        super().__init__(vmin, vmax, clip)

    def __call__(self, value, clip=None):
        # I'm ignoring masked values and all kinds of edge cases to make a
        # simple example...
        # Note also that we must extrapolate beyond vmin/vmax
        x, y = [self.vmin, self.vcenter, self.vmax], [0, 0.5, 1.]
        return np.ma.masked_array(np.interp(value, x, y,
                                            left=-np.inf, right=np.inf))

    def inverse(self, value):
        y, x = [self.vmin, self.vcenter, self.vmax], [0, 0.5, 1]
        return np.interp(value, x, y, left=-np.inf, right=np.inf)

def truncate_colormap(cmap, minval=0.0, maxval=1.0, n=100):
    new_cmap = colors.LinearSegmentedColormap.from_list(
        'trunc({n},{a:.2f},{b:.2f})'.format(n=cmap.name, a=minval, b=maxval),
        cmap(np.linspace(minval, maxval, n)))
    return new_cmap

In [ ]:
def make_metpycompliant(afile, polelat, polelon):
    #Unfortunately, metpy requires some specific dimension names, and also gets mad if
    #we have two different latitude/longitude coordinates, so we need to rename some
    #stuff in our post-processed files and get rid of the 1D lat/lon coords
    afile = afile.drop(["lat1d", "lon1d"]).rename({"z": "vertical", "lat2d": "lat", "lon2d": "lon"})
    afile = afile.metpy.assign_crs(grid_mapping_name = "stereographic", latitude_of_projection_origin = polelat, longitude_of_projection_origin = polelon, scale_factor_at_projection_origin = 1)
    return afile

In [ ]:
def add_cartofeatures_fulldomain(geoax):
    geoax.set_extent((controlgrid["x"].min(), controlgrid["x"].max(), controlgrid["y"].min(), controlgrid["y"].max()), crs = ramscrs)
    # ax.annotate(axlabels[i], (0.04, 0.94), xycoords = "axes fraction", color = "black", fontsize = 6.5, horizontalalignment = "center", bbox = {"edgecolor": "black", "linewidth": 1, "facecolor": "white"}, zorder = 2)
    geoax.add_feature(cfeature.NaturalEarthFeature('physical', 'coastline', '50m', edgecolor = 'darkgrey', facecolor = "none", linewidth = 0.4, zorder = 3))
    # ax.add_feature(cfeature.NaturalEarthFeature('cultural', 'admin_0_boundary_lines_land', '50m', edgecolor = "black", facecolor = "none", linewidth = 0.4, zorder = 3))
    # ax.add_feature(cfeature.NaturalEarthFeature('physical', 'lakes', '50m',
    #                                             edgecolor='navy',
    #                                             facecolor="dodgerblue", linewidth = 0.2), zorder = 1)
    # ax.add_feature(cfeature.NaturalEarthFeature(category='cultural', name='admin_1_states_provinces_lines', facecolor='none', scale = "50m", edgecolor='darkgrey', zorder = 2))
    geoax.add_feature(cfeature.NaturalEarthFeature(category="cultural", name = "admin_1_states_provinces_lakes", linewidth = 0.4, facecolor = "none", scale = "50m", edgecolor = "black", zorder = 3))
    
    gl = geoax.gridlines(crs = ccrs.PlateCarree(), draw_labels = {"bottom": "x", "left": "y"}, x_inline = False, y_inline = False, dms = True)
    gl.xlocator = mticker.FixedLocator([-83, -79, -75])
    gl.ylocator = mticker.FixedLocator([38, 40, 42, 44, 46])
    gl.bottom_labels = True
    gl.left_labels   = True
    gl.xlines = False
    gl.ylines = False
    gl.top_labels    = False
    gl.right_labels  = False
    gl.xlabel_style = {'size': 8, 'color': 'black', 'rotation': 0, "horizontalalignment": "right"}
    gl.ylabel_style = {'size': 8, 'color': 'black', 'rotation': 0, "horizontalalignment": "right"}
    # ax.set_ylabel("Latitude")
    # ax.set_xlabel("Longitude")
    return geoax

In [ ]:
def add_cartofeatures_zoomdomain(geoax):
    geoax.set_extent((controlgrid["x"][50], controlgrid["x"][650], controlgrid["y"][450], controlgrid["y"][1050]), crs = ramscrs)
    # ax.annotate(axlabels[i], (0.04, 0.94), xycoords = "axes fraction", color = "black", fontsize = 6.5, horizontalalignment = "center", bbox = {"edgecolor": "black", "linewidth": 1, "facecolor": "white"}, zorder = 2)
    geoax.add_feature(cfeature.NaturalEarthFeature('physical', 'coastline', '50m', edgecolor = 'darkgrey', facecolor = "none", linewidth = 0.4, zorder = 3))
    # ax.add_feature(cfeature.NaturalEarthFeature('cultural', 'admin_0_boundary_lines_land', '50m', edgecolor = "black", facecolor = "none", linewidth = 0.4, zorder = 3))
    # ax.add_feature(cfeature.NaturalEarthFeature('physical', 'lakes', '50m',
    #                                             edgecolor='navy',
    #                                             facecolor="dodgerblue", linewidth = 0.2), zorder = 1)
    # ax.add_feature(cfeature.NaturalEarthFeature(category='cultural', name='admin_1_states_provinces_lines', facecolor='none', scale = "50m", edgecolor='darkgrey', zorder = 2))
    geoax.add_feature(cfeature.NaturalEarthFeature(category="cultural", name = "admin_1_states_provinces_lakes", linewidth = 0.4, facecolor = "none", scale = "50m", edgecolor = "black", zorder = 3))
    
    gl = geoax.gridlines(crs = ccrs.PlateCarree(), draw_labels = {"bottom": "x", "left": "y"}, x_inline = False, y_inline = False, dms = True)
    gl.xlocator = mticker.FixedLocator([-84, -82, -80, -78])
    gl.ylocator = mticker.FixedLocator([42, 43, 44, 45, 46])
    gl.bottom_labels = True
    gl.left_labels   = True
    gl.xlines = False
    gl.ylines = False
    gl.top_labels    = False
    gl.right_labels  = False
    gl.xlabel_style = {'size': 8, 'color': 'black', 'rotation': 0, "horizontalalignment": "right"}
    gl.ylabel_style = {'size': 8, 'color': 'black', 'rotation': 0, "horizontalalignment": "right"}
    # ax.set_ylabel("Latitude")
    # ax.set_xlabel("Longitude")
    return geoax

In [ ]:
def add_cartofeatures_wmovement(geoax):
    geoax.set_extent((-84, -80, 41, 44), crs = ccrs.PlateCarree())
    # ax.annotate(axlabels[i], (0.04, 0.94), xycoords = "axes fraction", color = "black", fontsize = 6.5, horizontalalignment = "center", bbox = {"edgecolor": "black", "linewidth": 1, "facecolor": "white"}, zorder = 2)
    geoax.add_feature(cfeature.NaturalEarthFeature('physical', 'coastline', '50m', edgecolor = 'darkgrey', facecolor = "none", linewidth = 0.4, zorder = 3))
    # ax.add_feature(cfeature.NaturalEarthFeature('cultural', 'admin_0_boundary_lines_land', '50m', edgecolor = "black", facecolor = "none", linewidth = 0.4, zorder = 3))
    # ax.add_feature(cfeature.NaturalEarthFeature('physical', 'lakes', '50m',
    #                                             edgecolor='navy',
    #                                             facecolor="dodgerblue", linewidth = 0.2), zorder = 1)
    # ax.add_feature(cfeature.NaturalEarthFeature(category='cultural', name='admin_1_states_provinces_lines', facecolor='none', scale = "50m", edgecolor='darkgrey', zorder = 2))
    geoax.add_feature(cfeature.NaturalEarthFeature(category="cultural", name = "admin_1_states_provinces_lakes", linewidth = 0.4, facecolor = "none", scale = "50m", edgecolor = "black", zorder = 3))
    
    gl = geoax.gridlines(crs = ccrs.PlateCarree(), draw_labels = {"bottom": "x", "left": "y"}, x_inline = False, y_inline = False, dms = True)
    gl.xlocator = mticker.FixedLocator([-83, -82, -81])
    gl.ylocator = mticker.FixedLocator([42, 43])
    gl.bottom_labels = True
    gl.left_labels   = True
    gl.xlines = False
    gl.ylines = False
    gl.top_labels    = False
    gl.right_labels  = False
    gl.xlabel_style = {'size': 8, 'color': 'black', 'rotation': 0, "horizontalalignment": "right"}
    gl.ylabel_style = {'size': 8, 'color': 'black', 'rotation': 0, "horizontalalignment": "right"}
    # ax.set_ylabel("Latitude")
    # ax.set_xlabel("Longitude")
    return geoax

In [ ]:
def add_cartofeatures_brownlake(geoax):
    geoax.set_extent((-80.75, -78.75, 42, 44.5), crs = ccrs.PlateCarree())
    # ax.annotate(axlabels[i], (0.04, 0.94), xycoords = "axes fraction", color = "black", fontsize = 6.5, horizontalalignment = "center", bbox = {"edgecolor": "black", "linewidth": 1, "facecolor": "white"}, zorder = 2)
    geoax.add_feature(cfeature.NaturalEarthFeature('physical', 'coastline', '50m', edgecolor = 'darkgrey', facecolor = "none", linewidth = 0.4, zorder = 3))
    # ax.add_feature(cfeature.NaturalEarthFeature('cultural', 'admin_0_boundary_lines_land', '50m', edgecolor = "black", facecolor = "none", linewidth = 0.4, zorder = 3))
    # ax.add_feature(cfeature.NaturalEarthFeature('physical', 'lakes', '50m',
    #                                             edgecolor='navy',
    #                                             facecolor="dodgerblue", linewidth = 0.2), zorder = 1)
    # ax.add_feature(cfeature.NaturalEarthFeature(category='cultural', name='admin_1_states_provinces_lines', facecolor='none', scale = "50m", edgecolor='darkgrey', zorder = 2))
    geoax.add_feature(cfeature.NaturalEarthFeature(category="cultural", name = "admin_1_states_provinces_lakes", linewidth = 0.4, facecolor = "none", scale = "50m", edgecolor = "black", zorder = 3))
    
    gl = geoax.gridlines(crs = ccrs.PlateCarree(), draw_labels = {"bottom": "x", "left": "y"}, x_inline = False, y_inline = False, dms = True)
    gl.xlocator = mticker.FixedLocator([-81, -80, -79])
    gl.ylocator = mticker.FixedLocator([43, 44])
    gl.bottom_labels = True
    gl.left_labels   = True
    gl.xlines = False
    gl.ylines = False
    gl.top_labels    = False
    gl.right_labels  = False
    gl.xlabel_style = {'size': 8, 'color': 'black', 'rotation': 0, "horizontalalignment": "right"}
    gl.ylabel_style = {'size': 8, 'color': 'black', 'rotation': 0, "horizontalalignment": "right"}
    # ax.set_ylabel("Latitude")
    # ax.set_xlabel("Longitude")
    return geoax

In [ ]:
def add_cartofeatures_snowbox(geoax):
    geoax.set_extent((controlgrid["x"][200], controlgrid["x"][750], controlgrid["y"][150], controlgrid["y"][650]), crs = ramscrs)
    # ax.annotate(axlabels[i], (0.04, 0.94), xycoords = "axes fraction", color = "black", fontsize = 6.5, horizontalalignment = "center", bbox = {"edgecolor": "black", "linewidth": 1, "facecolor": "white"}, zorder = 2)
    geoax.add_feature(cfeature.NaturalEarthFeature('physical', 'coastline', '50m', edgecolor = 'darkgrey', facecolor = "none", linewidth = 0.4, zorder = 3))
    # ax.add_feature(cfeature.NaturalEarthFeature('cultural', 'admin_0_boundary_lines_land', '50m', edgecolor = "black", facecolor = "none", linewidth = 0.4, zorder = 3))
    # ax.add_feature(cfeature.NaturalEarthFeature('physical', 'lakes', '50m',
    #                                             edgecolor='navy',
    #                                             facecolor="dodgerblue", linewidth = 0.2), zorder = 1)
    # ax.add_feature(cfeature.NaturalEarthFeature(category='cultural', name='admin_1_states_provinces_lines', facecolor='none', scale = "50m", edgecolor='darkgrey', zorder = 2))
    geoax.add_feature(cfeature.NaturalEarthFeature(category="cultural", name = "admin_1_states_provinces_lakes", linewidth = 0.4, facecolor = "none", scale = "50m", edgecolor = "black", zorder = 3))
    
    gl = geoax.gridlines(crs = ccrs.PlateCarree(), draw_labels = {"bottom": "x", "left": "y"}, x_inline = False, y_inline = False, dms = True)
    gl.xlocator = mticker.FixedLocator([-82, -81, -80, -79, -78])
    gl.ylocator = mticker.FixedLocator([38, 39, 40, 41, 42])
    gl.bottom_labels = True
    gl.left_labels   = True
    gl.xlines = False
    gl.ylines = False
    gl.top_labels    = False
    gl.right_labels  = False
    gl.xlabel_style = {'size': 8, 'color': 'black', 'rotation': 0, "horizontalalignment": "right"}
    gl.ylabel_style = {'size': 8, 'color': 'black', 'rotation': 0, "horizontalalignment": "right"}
    # ax.set_ylabel("Latitude")
    # ax.set_xlabel("Longitude")
    return ax

## Establish directories for files for CONTROL and NLH

In [ ]:
controlpath = input("Enter the directory containing post-processed output from the CONTROL simulation: ").rstrip("/ ")
if not os.path.exists(controlpath):
    raise Exception("Control post-processed file directory not found!")
nlhpath = input("Enter the directory containing post-processed output from the NLH simulation: ").rstrip("/ ")
if not os.path.exists(nlhpath):
    raise Exception("NLH post-processed file directory not found!")

In [ ]:
gridsmade = input("Have you made grid files for the CONTROL and NLH simulations? These are just files which contain basic information about coordinates, patch areas, and terrain height. If yes, there should be NetCDF files in the directory with the data for each simulation with the name 'gridconsts.nc'. Enter 'yes' or 'no': ")
if gridsmade.lower().strip() == "yes":
    gridsmade = True
elif gridsmade.lower().strip() == "no":
    gridsmade = False
else:
    raise Exception("Answer must be either 'yes' or 'no'!")
if gridsmade:
    controlgridpath = input("Enter the directory containing the gridconsts.nc file for the CONTROL simulation: ").rstrip("/ ")
    nlhgridpath = input("Enter the directory containing the gridconsts.nc file for the NLH simulation: ").rstrip("/ ")
    if not os.path.exists(f"{controlgridpath}/gridconsts.nc"):
        raise Exception("Control grid file not found!")
    if not os.path.exists(f"{nlhgridpath}/gridconsts.nc"):
        raise Exception("NLH grid file not found!")
    controlgrid = xr.open_dataset(f"{controlgridpath}/gridconsts.nc")
    nlhgrid = xr.open_dataset(f"{nlhgridpath}/gridconsts.nc")
else:
    control_file_init = xr.open_dataset(f"{controlpath}/mvars-cart-2010-01-02-180000-g1.nc")
    nlh_file_init = xr.open_dataset(f"{nlhpath}/mvars-cart-2010-01-02-180000-g1.nc")
    controlgrid = control_file_init[["PatchArea", "Topo"]]
    nlhgrid = nlh_file_init[["PatchArea", "Topo"]]
    control_file_init.close(); del control_file_init
    nlh_file_init.close(); del nlh_file_init
    controlgridpath = input("Enter the directory where you want to put the gridconsts.nc file for the CONTROL simulation: ").rstrip("/ ")
    nlhgridpath = input("Enter the directory where you want to put the gridconsts.nc file for the NLH simulation: ").rstrip("/ ")
    controlgrid.to_netcdf(f"{controlgridpath}/gridconsts.nc")
    nlhgrid.to_netcdf(f"{nlhgridpath}/gridconsts.nc")

In [ ]:
varinit_control = xr.open_dataset(f"{controlgridpath}/varfiles/var-V-2010-01-01-120000-g1.h5")
print(varinit_control.dims)
varinit_control = varinit_control.isel(phony_dim_1 = slice(1,1099), phony_dim_2 = slice(1, 999), phony_dim_0 = 1)
print(varinit_control)
varinit_nolake = xr.open_dataset(f"{nlhgridpath}/varfiles/var-V-2010-01-01-120000-g1.h5")
varinit_nolake = varinit_nolake.isel(phony_dim_1 = slice(1,1099), phony_dim_2 = slice(1, 999), phony_dim_0 = 1)

control_snowinitmask_full = np.where(controlgrid["PatchArea"].isel(patch = 1) == 1, varinit_control["SNOWMASS"].values, np.nan); print(control_snowinitmask_full.shape)
nolake_snowinitmask_full = np.where(nlhgrid["PatchArea"].isel(patch = 1) == 1, varinit_nolake["SNOWMASS"].values, np.nan); print(nolake_snowinitmask_full.shape)

## Make a figure of the simulation domains (terrain height and water body location) as well as surface temperature and snow cover initialization

In [ ]:
lhymin = 640; lhymax = 1040; lhxmin = 140; lhxmax = 420 #lh - main body of Lake Huron
gbymin = 800; gbymax = 1000; gbxmin = 420; gbxmax = 700 #gb - Georgian Bay and downwind land
gbpatchpoints = np.zeros((4, 2))
gbpatchpoints[:,0] = [controlgrid["x"].values[gbxmin], controlgrid["x"].values[gbxmax], controlgrid["x"].values[gbxmax], controlgrid["x"].values[gbxmin]]
gbpatchpoints[:,1] = [controlgrid["y"].values[gbymin], controlgrid["y"].values[gbymin], controlgrid["y"].values[gbymax], controlgrid["y"].values[gbymax]]

lhpatchpoints = np.zeros((4, 2))
lhpatchpoints[:,0] = [controlgrid["x"].values[lhxmin], controlgrid["x"].values[lhxmax], controlgrid["x"].values[lhxmax], controlgrid["x"].values[lhxmin]]
lhpatchpoints[:,1] = [controlgrid["y"].values[lhymin], controlgrid["y"].values[lhymin], controlgrid["y"].values[lhymax], controlgrid["y"].values[lhymax]]

### Do the same thing, but now with hillshading

In [ ]:
from matplotlib.colors import LightSource
hillshadegreycmap = mpl.colormaps.get_cmap("Greys")(np.linspace(0, 1, 256))
hillshadegreycmap[:,3] = abs(np.linspace(-0.5, 0.5, 256))
hillshadegreycmap = mpl.colors.ListedColormap(hillshadegreycmap)
print(hillshadegreycmap)
ls = LightSource (azdeg = 45, altdeg = 20)
topomp = ls.hillshade(controlgrid["Topo"].values, dx = 1000, dy = 1000, vert_exag = 50)

# terls = colors.LightSource(azdeg = 315, altdeg = 30)
# statesgeom = shapely.read_file("cb_2018_us_state_20m/cb_2018_us_state_20m.shp")
ramscrs = ccrs.Stereographic(central_longitude = -80, central_latitude = 42)
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize = (5, 7), sharex = True, sharey = True, dpi = 300, subplot_kw = {"projection": ramscrs}, layout = "compressed")
axlabels = ["(a)", "(b)", "(c)", "(d)"]
modtopocmap = truncate_colormap(mpl.colormaps.get_cmap("gist_earth"), minval = 0.5, maxval = 1, n = 128)
for i, ax in enumerate(fig.get_axes()):
    # ax.set_extent((controlgrid["lon2d"].min(), controlgrid["lon2d"].max(), controlgrid["lat2d"].min()-2, controlgrid["lat2d"].max()), crs = ccrs.PlateCarree())
    ax = add_cartofeatures_fulldomain(ax)
    left, width = 0, 0.1
    bottom, height = 0.90, 0.1
    right = left + width
    top = bottom + height
    p = plt.Rectangle((left, bottom), width, height, fill=True, zorder = 5, edgecolor = "black", linewidth = 0.2, facecolor = "white")
    p.set_transform(ax.transAxes)
    p.set_clip_on(False)
    ax.add_patch(p)
    ax.text(left+0.5*width, bottom+0.5*height, axlabels[i], fontsize = 10, transform = ax.transAxes, horizontalalignment = "center", verticalalignment = "center", zorder = 6)
termp = ax1.pcolormesh(controlgrid["x"], controlgrid["y"], controlgrid["Topo"], cmap = modtopocmap, vmin = 0, vmax = 1000, transform = ramscrs, zorder = 0)
ax1.pcolormesh(controlgrid["x"], controlgrid["y"], topomp, cmap = hillshadegreycmap, vmin = 0, vmax = 1, transform = ramscrs, zorder = 2)
ax2.pcolormesh(controlgrid["x"], controlgrid["y"], nlhgrid["Topo"], cmap = modtopocmap, vmin = 0, vmax = 1000, transform = ramscrs)
ax2.pcolormesh(controlgrid["x"], controlgrid["y"], topomp, cmap = hillshadegreycmap, vmin = 0, vmax = 1, transform = ramscrs, zorder = 2)
ax1.pcolormesh(controlgrid["x"], controlgrid["y"], controlgrid["PatchArea"][0,:,:].where(controlgrid["PatchArea"][0,:,:]>0.01), vmin = 0, vmax = 0.1, cmap = "coolwarm_r", shading = "nearest", transform = ramscrs, zorder = 1)
ax2.pcolormesh(controlgrid["x"], controlgrid["y"], nlhgrid["PatchArea"][0,:,:].where(nlhgrid["PatchArea"][0,:,:]>0.01), vmin = 0, vmax = 0.1, cmap = "coolwarm_r", shading = "nearest", transform = ramscrs, zorder = 1)
# ax2.plot([controlgrid["x"][lhxmin], controlgrid["x"][lhxmax]], [controlgrid["y"][lhymin], controlgrid["y"][lhymin]], color = "red", transform = ramscrs)
ax1.annotate("Lake Huron", [-82.7, 44.6], horizontalalignment = "center", verticalalignment = "center", fontfamily = "sans", fontweight = "demibold", fontsize = 6.5, rotation = 290, color = "white", transform = ccrs.PlateCarree(), zorder = 3)
ax1.annotate("Lake Erie", [-81.2, 42.1], horizontalalignment = "center", verticalalignment = "center", fontfamily = "sans", fontweight = "demibold", fontsize = 6.5, rotation = 23, color = "white", transform = ccrs.PlateCarree(), zorder = 3)
ax1.annotate("Lake Ont.", [-78, 43.6], horizontalalignment = "center", verticalalignment = "center", fontfamily = "sans", fontweight = "demibold", fontsize = 6.5, rotation = 0, color = "white", transform = ccrs.PlateCarree(), zorder = 3)
ax1.annotate("GB", [-80.7, 45.1], horizontalalignment = "center", verticalalignment = "center", fontfamily = "sans", fontweight = "demibold", fontsize = 6.5, rotation = 0, color = "white", transform = ccrs.PlateCarree(), zorder = 3)
ax2.add_patch(mpl.patches.Polygon(gbpatchpoints, edgecolor = "firebrick", facecolor = [1,0,0,0], linewidth = 0.6, zorder = 3, transform = ramscrs))
ax2.add_patch(mpl.patches.Polygon(lhpatchpoints, edgecolor = "firebrick", facecolor = [1,0,0,0], linewidth = 0.6, zorder = 3, transform = ramscrs))
# ax2.annotate("Box \n LH", [-82.5, 44.8], horizontalalignment = "center", verticalalignment = "center", fontfamily = "sans", fontweight = "demibold", fontsize = 10, color = "red", transform = ccrs.PlateCarree(), zorder = 4)
# ax2.annotate("Box \n GB", [-79.3, 45.1], horizontalalignment = "center", verticalalignment = "center", fontfamily = "sans", fontweight = "demibold", fontsize = 10, color = "red", transform = ccrs.PlateCarree(), zorder = 4)
# ax2.annotate("3", [-79, 40], horizontalalignment = "center", verticalalignment = "center", fontfamily = "sans", fontweight = "demibold", fontsize = 10, color = "cyan", transform = ccrs.PlateCarree())
ax2.annotate("3", [-79.5, 42], horizontalalignment = "center", verticalalignment = "center", fontfamily = "sans", fontweight = "demibold", fontsize = 10, color = "cyan", transform = ccrs.PlateCarree())
ax2.annotate("2", [-80.5, 43.5], horizontalalignment = "center", verticalalignment = "center", fontfamily = "sans", fontweight = "demibold", fontsize = 10, color = "cyan", transform = ccrs.PlateCarree())
ax2.annotate("1", [-82.3, 45.8], horizontalalignment = "center", verticalalignment = "center", fontfamily = "sans", fontweight = "demibold", fontsize = 10, color = "cyan", transform = ccrs.PlateCarree())

ax1.set_title("CONTROL"); ax3.set_title("CONTROL")
ax2.set_title("NLH"); ax4.set_title("NLH")
# ax1.set_ylabel("Latitude (Degrees N)", fontfamily = "arial")
tercbar = fig.colorbar(termp, ax = [ax1, ax2], orientation = "horizontal", ticks = [0, 250, 500, 750, 1000], fraction = 0.05, extend = "max"); tercbar.set_label("Terrain Height (m)")
from palettable.lightbartlein.sequential import Blues10_10 as snowpalette
snowcmap = snowpalette.get_mpl_colormap().reversed()
from palettable.cartocolors.sequential import agSunset_7 as temppalette
tempcmap = temppalette.get_mpl_colormap()
tempmp = ax3.pcolormesh(controlgrid["x"], controlgrid["y"], varinit_control["SOILTEMP1"], vmin = 270, vmax = 280, cmap = tempcmap, transform = ramscrs, zorder = 0)
snowmp1 = ax3.contour(controlgrid["x"], controlgrid["y"], varinit_control["SNOWMASS"], levels = [2, 5, 10, 20, 30, 50], cmap = snowcmap, norm = "log", zorder = 1, transform = ramscrs)
ax3.pcolormesh(controlgrid["x"], controlgrid["y"], controlgrid["PatchArea"].isel(patch = 0).where(controlgrid["PatchArea"].isel(patch = 0) == 1), vmin = 0, vmax = 1, cmap = "Greys", zorder = 2, transform = ramscrs)
snowmp1.clabel(inline = True, fontsize = 6, zorder = 4)
ax4.pcolormesh(controlgrid["x"], controlgrid["y"], varinit_nolake["SOILTEMP1"], vmin = 270, vmax = 280, cmap = tempcmap, transform = ramscrs)
snowmp2 = ax4.contour(nlhgrid["x"], nlhgrid["y"], nolake_snowinitmask_full, levels = [2, 5, 10, 20, 30, 50], cmap = snowcmap, norm = "log")
snowmp2.clabel(inline = True, fontsize = 6, zorder = 4)
ax4.pcolormesh(nlhgrid["x"], nlhgrid["y"], nlhgrid["PatchArea"].isel(patch = 0).where(nlhgrid["PatchArea"].isel(patch = 0) == 1), vmin = 0, vmax = 1, cmap = "Greys", zorder = 1, transform = ramscrs)
tempcbar = fig.colorbar(tempmp, ax = [ax3,ax4], orientation = "horizontal", fraction = 0.1, extend = "both", aspect = 35, label = "Soil Temperature (K)")

# ax1.set_xlabel("Longitude (Degrees E)", fontsize = 20, fontfamily = "arial")
# ax2.set_xlabel("Longitude (Degrees E)", fontsize = 20, fontfamily = "arial")
# # fig.text(0.5, 0.18, "Longitude (Degrees E)", fontsize = 20, ha = "center", name = "Liberation Serif")
# fig.suptitle("Simulation Domain", fontfamily = "Arial")
fig.savefig("../PaperFigs/domaincarto-hillshade_4panel.png")
plt.close()
del fig; del ax1; del ax2; del ax3; del ax4

### Previous 2-Panel version of domain comparison without hillshading

In [ ]:
# terls = colors.LightSource(azdeg = 315, altdeg = 30)
# statesgeom = shapely.read_file("cb_2018_us_state_20m/cb_2018_us_state_20m.shp")
ramscrs = ccrs.Stereographic(central_longitude = -80, central_latitude = 42)
fig, (ax1, ax2) = plt.subplots(1, 2, figsize = (5, 3.5), sharex = True, sharey = True, dpi = 300, subplot_kw = {"projection": ramscrs})
axlabels = ["(a)", "(b)"]
modtopocmap = truncate_colormap(mpl.colormaps.get_cmap("gist_earth"), minval = 0.5, maxval = 1, n = 128)
for i, ax in enumerate(fig.get_axes()):
    # ax.set_extent((controlgrid["lon2d"].min(), controlgrid["lon2d"].max(), controlgrid["lat2d"].min()-2, controlgrid["lat2d"].max()), crs = ccrs.PlateCarree())
    ax = add_cartofeatures_fulldomain(ax)
    left, width = 0, 0.1
    bottom, height = 0.90, 0.1
    right = left + width
    top = bottom + height
    p = plt.Rectangle((left, bottom), width, height, fill=True, zorder = 3, edgecolor = "black", linewidth = 0.2, facecolor = "white")
    p.set_transform(ax.transAxes)
    p.set_clip_on(False)
    ax.add_patch(p)
    ax.text(left+0.5*width, bottom+0.5*height, axlabels[i], fontsize = 10, transform = ax.transAxes, horizontalalignment = "center", verticalalignment = "center")
termp = ax1.pcolormesh(controlgrid["x"], controlgrid["y"], controlgrid["Topo"], cmap = modtopocmap, vmin = 0, vmax = 1000, transform = ramscrs, zorder = 0)
ax2.pcolormesh(controlgrid["x"], controlgrid["y"], nlhgrid["Topo"], cmap = modtopocmap, vmin = 0, vmax = 1000, transform = ramscrs)
ax1.pcolormesh(controlgrid["x"], controlgrid["y"], controlgrid["PatchArea"][0,:,:].where(controlgrid["PatchArea"][0,:,:]>0.01), vmin = 0, vmax = 0.1, cmap = "coolwarm_r", shading = "nearest", transform = ramscrs)
ax2.pcolormesh(controlgrid["x"], controlgrid["y"], nlhgrid["PatchArea"][0,:,:].where(nlhgrid["PatchArea"][0,:,:]>0.01), vmin = 0, vmax = 0.1, cmap = "coolwarm_r", shading = "nearest", transform = ramscrs)
ax2.plot([controlgrid["x"][lhxmin], controlgrid["x"][lhxmax]], [controlgrid["y"][lhymin], controlgrid["y"][lhymin]], color = "red", transform = ramscrs)
ax1.annotate("Lake Huron", [-82.7, 44.6], horizontalalignment = "center", verticalalignment = "center", fontfamily = "sans", fontweight = "demibold", fontsize = 6.5, rotation = 290, color = "white", transform = ccrs.PlateCarree())
ax1.annotate("Lake Erie", [-81.2, 42.1], horizontalalignment = "center", verticalalignment = "center", fontfamily = "sans", fontweight = "demibold", fontsize = 6.5, rotation = 23, color = "white", transform = ccrs.PlateCarree())
ax1.annotate("Lake Ont.", [-78, 43.6], horizontalalignment = "center", verticalalignment = "center", fontfamily = "sans", fontweight = "demibold", fontsize = 6.5, rotation = 0, color = "white", transform = ccrs.PlateCarree())
ax1.annotate("GB", [-80.7, 45.1], horizontalalignment = "center", verticalalignment = "center", fontfamily = "sans", fontweight = "demibold", fontsize = 6.5, rotation = 0, color = "white", transform = ccrs.PlateCarree())
ax2.add_patch(mpl.patches.Polygon(gbpatchpoints, edgecolor = "firebrick", facecolor = [1,0,0,0], linewidth = 0.6, zorder = 3, transform = ramscrs))
ax2.add_patch(mpl.patches.Polygon(lhpatchpoints, edgecolor = "firebrick", facecolor = [1,0,0,0], linewidth = 0.6, zorder = 3, transform = ramscrs))
ax2.annotate("Box \n LH", [-82.5, 44.8], horizontalalignment = "center", verticalalignment = "center", fontfamily = "sans", fontweight = "demibold", fontsize = 10, color = "red", transform = ccrs.PlateCarree())
# ax2.annotate("Box \n GB", [-79.3, 45.1], horizontalalignment = "center", verticalalignment = "center", fontfamily = "sans", fontweight = "demibold", fontsize = 10, color = "red", transform = ccrs.PlateCarree())
# ax2.annotate("3", [-79, 40], horizontalalignment = "center", verticalalignment = "center", fontfamily = "sans", fontweight = "demibold", fontsize = 10, color = "cyan", transform = ccrs.PlateCarree())
ax2.annotate("3", [-79.5, 42], horizontalalignment = "center", verticalalignment = "center", fontfamily = "sans", fontweight = "demibold", fontsize = 10, color = "cyan", transform = ccrs.PlateCarree())
ax2.annotate("2", [-80.5, 43.5], horizontalalignment = "center", verticalalignment = "center", fontfamily = "sans", fontweight = "demibold", fontsize = 10, color = "cyan", transform = ccrs.PlateCarree())
ax2.annotate("1", [-82.3, 45.8], horizontalalignment = "center", verticalalignment = "center", fontfamily = "sans", fontweight = "demibold", fontsize = 10, color = "cyan", transform = ccrs.PlateCarree())

ax1.set_title("CONTROL", fontfamily = "Arial")
ax2.set_title("NLH", fontfamily = "Arial")
# ax1.set_ylabel("Latitude (Degrees N)", fontfamily = "arial")
cbar = fig.colorbar(termp, ax = [ax1, ax2], orientation = "horizontal", ticks = [0, 250, 500, 750, 1000], fraction = 0.05, extend = "max"); cbar.set_label("Terrain Height (m)", fontfamily = "Arial")

# ax1.set_xlabel("Longitude (Degrees E)", fontsize = 20, fontfamily = "arial")
# ax2.set_xlabel("Longitude (Degrees E)", fontsize = 20, fontfamily = "arial")
# # fig.text(0.5, 0.18, "Longitude (Degrees E)", fontsize = 20, ha = "center", name = "Liberation Serif")
# fig.suptitle("Simulation Domain", fontfamily = "Arial")
fig.savefig("../PaperFigs/domaincarto_2panel.png")
plt.close()
del fig; del ax1; del ax2

## 4-Panel plot showing soil/snow initialization

In [ ]:
moresnowbasepath = input("Enter the path to the base directory of the MORESNOW simulation").strip()
varinit_moresnow = xr.open_dataset(f"{moresnowbasepath}/varfiles/var-V-2010-01-01-120000-g1.h5")
varinit_moresnow = varinit_moresnow.isel(phony_dim_1 = slice(1,1099), phony_dim_2 = slice(1, 999), phony_dim_0 = 1)
print(varinit_moresnow)

moresnow_snowinitmask_full = np.where(controlgrid["PatchArea"].isel(patch = 1) == 1, varinit_moresnow["SNOWMASS"].values, np.nan); print(moresnow_snowinitmask_full.shape)

In [ ]:
print(np.where(controlgrid["PatchArea"].isel(x = slice(50, 650), y = slice(450, 1050), patch = 1) == 1, varinit_moresnow["SNOWMASS"].isel(phony_dim_1 = slice(451, 1051), phony_dim_2 = slice(51, 651)), np.nan))

In [ ]:
print(varinit_moresnow["SNOWMASS"].isel(phony_dim_2 = 451, phony_dim_1 = 792).values)

In [ ]:
plt.figure()
plt.pcolormesh(moresnow_snowinitmask, cmap = "Blues", vmin = 0, vmax = 30)
plt.axvline(400, color = "black")
plt.axhline(250, color = "black")

## 2-Panel plot showing CONTROL vs MORESNOW initialization

In [ ]:
control_snowinitmask = np.where(controlgrid["PatchArea"].isel(x = slice(50, 650), y = slice(450, 1050), patch = 1) == 1, varinit_control["SNOWMASS"].isel(phony_dim_2 = slice(51, 651), phony_dim_1 = slice(451, 1051)), np.nan); print(control_snowinitmask.shape)
moresnow_snowinitmask = np.where(controlgrid["PatchArea"].isel(x = slice(50, 650), y = slice(450, 1050), patch = 1) == 1, varinit_moresnow["SNOWMASS"].isel(phony_dim_2 = slice(51, 651), phony_dim_1 = slice(451, 1051)), np.nan); print(moresnow_snowinitmask.shape)
ramscrs = ccrs.Stereographic(central_longitude = -80, central_latitude = 42)
fig, (ax1, ax2) = plt.subplots(1,2, figsize = (5, 3.5), dpi = 200, subplot_kw = {"projection": ramscrs}, layout = "compressed")
axlabels = ["(a)", "(b)"]
for i, ax in enumerate(fig.get_axes()):
    # termp = ax.pcolormesh(controlgrid["x"].isel(x = slice(50, 650)), controlgrid["y"].isel(y = slice(450, 1050)), controlgrid["Topo"].isel(x = slice(50, 650), y = slice(450, 1050)), vmin = 0, vmax = 1000, cmap = modtopocmap, transform = ramscrs)
    # ax.set_extent((controlgrid["lon2d"].min(), controlgrid["lon2d"].max(), controlgrid["lat2d"].min()-2, controlgrid["lat2d"].max()), crs = ccrs.PlateCarree())
    ax = add_cartofeatures_zoomdomain(ax)
    left, width = 0, 0.1
    bottom, height = 0.90, 0.1
    right = left + width
    top = bottom + height
    p = plt.Rectangle((left, bottom), width, height, fill=True, zorder = 3, edgecolor = "black", linewidth = 0.2, facecolor = "white")
    p.set_transform(ax.transAxes)
    p.set_clip_on(False)
    ax.add_patch(p)
    ax.text(left+0.5*width, bottom+0.5*height, axlabels[i], fontsize = 10, transform = ax.transAxes, horizontalalignment = "center", verticalalignment = "center")
fig.suptitle("SWE Initialization")
ax1.set_title("CONTROL")
ax2.set_title("MORESNOW")
snowmp = ax1.pcolormesh(controlgrid["x"].isel(x = slice(50, 650)), controlgrid["y"].isel(y = slice(450, 1050)), control_snowinitmask, cmap = "Blues", vmin = 0, vmax = 25, norm = "linear", transform = ramscrs)
ax2.pcolormesh(controlgrid["x"].isel(x = slice(50, 650)), controlgrid["y"].isel(y = slice(450, 1050)), moresnow_snowinitmask, cmap = "Blues", vmin = 0, vmax = 25, norm = "linear", transform = ramscrs)
cbar = fig.colorbar(snowmp, ax = [ax1, ax2], orientation = "horizontal", fraction = 0.05, extend = "both", aspect = 25, label = "Snow Water Equivalent (mm)")
cbar.ax.axvline(moresnow_snowinitmask[250,400], color = "black", linestyle = (-0.5, (1,1)), linewidth = 1)
fig.savefig("../PaperFigs/supplemental_moresnowswecomp.png")
plt.close()
del fig; del ax1; del ax2; del cbar

## Calculating/Plotting Changes in Snowfall Between CONTROL and NLH

In [ ]:
nolake_snow_old = xr.open_dataset(f"{nlhpath}/mvars-cart-2010-01-02-060000-g1.nc")
control_snow_old = xr.open_dataset(f"{controlpath}/mvars-cart-2010-01-02-060000-g1.nc")
nolake_snow_new = xr.open_dataset(f"{nlhpath}/mvars-cart-2010-01-03-060000-g1.nc")
control_snow_new = xr.open_dataset(f"{controlpath}/mvars-cart-2010-01-03-060000-g1.nc")

snowdiff_control = control_snow_new["SnowPrecipTotal"]+control_snow_new["AggPrecipTotal"]-control_snow_old["SnowPrecipTotal"]-control_snow_old["AggPrecipTotal"]
snowdiff_nolake = nolake_snow_new["SnowPrecipTotal"]+nolake_snow_new["AggPrecipTotal"]-nolake_snow_old["SnowPrecipTotal"]-nolake_snow_old["AggPrecipTotal"]

### Define boxes for evaluating snowfall in Regions A,B,C,D,E

In [ ]:
ix0 = 400; iy0 = 525 #Northwest side of the "downwind of lake erie" box, at the center of the downwind shore
x0 = snowdiff_control["x"].isel(x = ix0); y0 = snowdiff_control["y"].isel(y = iy0); dxy = np.diff(snowdiff_control["x"])[0]
width = 400000; length = 305000 #m, dimensions of the box
tarray = np.arange(0, length+1, dxy); narray = np.arange(-width/2, width/2+1, dxy) #Coordinates of the box in tangent (long-axis) and normal (short-axis) directions. Normal is negative and positive (northeast and southwest), while tangent is only positive (to the right)
tmesh, nmesh = np.meshgrid(tarray, narray)
angle = np.pi/180*(-60) #Angle of the long axis of the box, with respect to the horizontal
xbox = x0.values+np.cos(angle)*tmesh-np.sin(angle)*nmesh
ybox = y0.values+np.sin(angle)*tmesh+np.cos(angle)*nmesh
snowbox = xr.DataArray(coords = {"y": (("normal", "tangent"), ybox), "x": (("normal", "tangent"), xbox), "normal": narray, "tangent": tarray}, dims = ["normal", "tangent"])

downwindsnow_control = snowdiff_control.interp(x = snowbox["x"], y = snowbox["y"])
downwindsnow_nolake = snowdiff_nolake.interp(x = snowbox["x"], y = snowbox["y"])

print(f"Average liquid equivalent of snowfall (mm) downwind of Lake Erie (Region A) in the CONTROl simulation \n{downwindsnow_control.mean():.2f} \nAverage liquid equivalent of snowfall downwind of Lake Erie (Region A) in the NLH simulation \n{downwindsnow_nolake.mean():.2f} \nRelative change in average snowfall downwind of Lake Erie (Region A) between CONTROL and NLH simulations \n{100*(downwindsnow_nolake.mean()-downwindsnow_control.mean())/downwindsnow_control.mean():.2f}% \nAverage CONTROL simulation snowfall downwind of Lake Erie (Region A) is {100*downwindsnow_control.mean()/downwindsnow_nolake.mean():.2f}% of average NLH simulation snowfall.")
print("\n\n")

patchpoints = np.zeros((4, 2))
patchpoints[:,0] = [downwindsnow_control["x"].values[0,0], downwindsnow_control["x"].values[0, -1], downwindsnow_control["x"].values[-1, -1], downwindsnow_control["x"].values[-1, 0]]
patchpoints[:,1] = [downwindsnow_control["y"].values[0,0], downwindsnow_control["y"].values[0, -1], downwindsnow_control["y"].values[-1, -1], downwindsnow_control["y"].values[-1, 0]]

print(f"Average liquid equivalent of snowfall in the main band (Region B) in CONTROL simulation \n{downwindsnow_control[210:250, 0:100].mean():.2f} \nAverage liquid equivalent of snowfall in the 'main band' (Region B) in NLH simulation \n{downwindsnow_nolake[210:250, 0:100].mean():.2f}\nRelative difference in average snowfall in the main band (Region B) between CONTROL and NLH simulation \n{100*(downwindsnow_control[210:250, 0:100].mean()-downwindsnow_nolake[210:250, 0:100].mean())/downwindsnow_control[210:250, 0:100].mean():.2f}% \nAverage CONTROL simulation snowfall in the main band (Region B) is {100*downwindsnow_control[210:250, 0:100].mean()/downwindsnow_nolake[210:250, 0:100].mean():.2f}% of average NLH simulation snowfall")
print("\n\n")
#Points 210:250, 0:100 form a rectangle roughly bounding the main snow band over Lake Erie in the CONTROL simulation
patchpointssub = np.zeros((4, 2)) #These are the points for box B in the figure below, outlining the main band
patchpointssub[:,0] = [downwindsnow_control["x"].values[210,0], downwindsnow_control["x"].values[210, 100], downwindsnow_control["x"].values[250, 100], downwindsnow_control["x"].values[250, 0]]
patchpointssub[:,1] = [downwindsnow_control["y"].values[210,0], downwindsnow_control["y"].values[210, 100], downwindsnow_control["y"].values[250, 100], downwindsnow_control["y"].values[250, 0]]
# print(downwindsnow_control[210:250, 0:100].mean(), downwindsnow_nolake[210:250, 0:100].mean(), downwindsnow_control[210:250, 0:100].sum()/downwindsnow_nolake[210:250, 0:100].sum())
print(f"Average liquid equivalent of snowfall downwind of NE Lake Erie (Region D) in CONTROL simulation \n{downwindsnow_control[350:400, 0:100].mean():.2f} \nAverage liquid equivalent of snowfall downwind of NE Lake Erie (Region D) in NLH simulation \n{downwindsnow_nolake[350:400, 0:100].mean():.2f}\nRelative difference in average snowfall downwind of NE Lake Erie (Region D) between CONTROL and NLH simulation \n{100*(downwindsnow_control[350:400, 0:100].mean()-downwindsnow_nolake[350:400, 0:100].mean())/downwindsnow_control[350:400, 0:100].mean():.2f}% \nAverage CONTROL simulation snowfall downwind of NE Lake Erie (Region D) is {100*downwindsnow_control[350:400, 0:100].mean()/downwindsnow_nolake[350:400, 0:100].mean():.2f}% of average NLH simulation snowfall")
#Change in snowfall in ne Lake Erie (Region D)
#Points 350:4000, 0:100 form a rectangle roughly bounding the region of snowfall from the "brown lake" effect in NE Lake Erie
print("\n\n")
patchpointsne = np.zeros((4,2)) #Points bounding the band in northeast Lake Erie (box D)
patchpointsne[:,0] = [downwindsnow_control["x"].values[350,0], downwindsnow_control["x"].values[350, 100], downwindsnow_control["x"].values[400, 100], downwindsnow_control["x"].values[400, 0]]
patchpointsne[:,1] = [downwindsnow_control["y"].values[350,0], downwindsnow_control["y"].values[350, 100], downwindsnow_control["y"].values[400, 100], downwindsnow_control["y"].values[400, 0]]

patchpoints_between = np.zeros((4, 2)) #Points bounding area downwind of Lake Erie between box B and box D (box C)
patchpoints_between[:,0] = [downwindsnow_control["x"].values[250,0], downwindsnow_control["x"].values[250, 100], downwindsnow_control["x"].values[350, 100], downwindsnow_control["x"].values[350, 0]]
patchpoints_between[:,1] = [downwindsnow_control["y"].values[250,0], downwindsnow_control["y"].values[250, 100], downwindsnow_control["y"].values[350, 100], downwindsnow_control["y"].values[350, 0]]

print(f"Average liquid equivalent of snowfall in Region C in CONTROL simulation \n{downwindsnow_control[250:350, 0:100].mean():.2f} \nAverage liquid equivalent of snowfall in Region C in NLH simulation \n{downwindsnow_nolake[250:350, 0:100].mean():.2f}\nRelative difference in average snowfall in Region C between CONTROL and NLH simulation \n{100*(downwindsnow_control[250:350, 0:100].mean()-downwindsnow_nolake[250:350, 0:100].mean())/downwindsnow_control[250:350, 0:100].mean():.2f}% \nAverage CONTROL simulation snowfall in Region C is {100*downwindsnow_control[250:350, 0:100].mean()/downwindsnow_nolake[250:350, 0:100].mean():.2f}% of average NLH simulation snowfall")

### Plot Snowfall Comparison between CONTROL and NLH

In [ ]:
ramscrs = ccrs.Stereographic(central_longitude = -80, central_latitude = 42)
fig, ((ax11, ax12, ax13), (ax21, ax22, ax23)) = plt.subplots(2, 3, figsize = (7.8, 6), dpi = 200, layout = "compressed", subplot_kw = {"projection": ramscrs})
startt = datetime(2010, 1, 2, 6, 0, 0); print(startt)
endt = datetime(2010, 1, 3, 6, 0, 0); print(endt)
axlabelstop = ["(a)", "(b)", "(c)"]
modtopocmap = truncate_colormap(mpl.colormaps.get_cmap("gist_earth"), minval = 0.5, maxval = 1, n = 128)
for i, ax in enumerate((ax11, ax12, ax13)):
    ax = add_cartofeatures_fulldomain(ax)
    left, width = 0, 0.1
    bottom, height = 0.90, 0.1
    right = left + width
    top = bottom + height
    p = plt.Rectangle((left, bottom), width, height, fill=True, zorder = 3, edgecolor = "black", linewidth = 0.2, facecolor = "white")
    p.set_transform(ax.transAxes)
    p.set_clip_on(False)
    ax.add_patch(p)
    ax.text(left+0.5*width, bottom+0.5*height, axlabelstop[i], fontsize = 10, transform = ax.transAxes, horizontalalignment = "center", verticalalignment = "center")
    termp = ax.pcolormesh(control_snow_old["x"], control_snow_old["y"], control_snow_old["Topo"], cmap = modtopocmap, vmin = 0, vmax = 1000, zorder = 0, transform = ramscrs)
ax13.add_patch(mpl.patches.Polygon(patchpoints, edgecolor = "black", facecolor = [1, 0, 0, 0], linewidth = 0.6, zorder = 3, transform = ramscrs))
ax13.add_patch(mpl.patches.Polygon(patchpointssub, edgecolor = "black", facecolor = [1,0,0,0], linewidth = 0.6, zorder = 3, transform = ramscrs))
ax13.add_patch(mpl.patches.Polygon(patchpointsne, edgecolor = "black", facecolor = [1,0,0,0], linewidth = 0.6, zorder = 3, transform = ramscrs))
ax13.add_patch(mpl.patches.Polygon(patchpoints_between, edgecolor = "black", facecolor = [1,0,0,0], linewidth = 0.6, zorder = 3, transform = ramscrs))
axlabelsbot = ["(d)", "(e)", "(f)"]
for i, ax in enumerate((ax21, ax22, ax23)):
    ax = add_cartofeatures_snowbox(ax)
    left, width = 0, 0.1
    bottom, height = 0.90, 0.1
    right = left + width
    top = bottom + height
    p = plt.Rectangle((left, bottom), width, height, fill=True, zorder = 3, edgecolor = "black", linewidth = 0.2, facecolor = "white")
    p.set_transform(ax.transAxes)
    p.set_clip_on(False)
    ax.add_patch(p)
    ax.text(left+0.5*width, bottom+0.5*height, axlabelsbot[i], fontsize = 10, transform = ax.transAxes, horizontalalignment = "center", verticalalignment = "center")
    ax.add_patch(mpl.patches.Polygon(patchpoints, edgecolor = "black", facecolor = [1, 0, 0, 0], linewidth = 0.6, zorder = 3, transform = ramscrs))
    ax.add_patch(mpl.patches.Polygon(patchpointssub, edgecolor = "black", facecolor = [1,0,0,0], linewidth = 0.6, zorder = 3, transform = ramscrs))
    ax.add_patch(mpl.patches.Polygon(patchpointsne, edgecolor = "black", facecolor = [1,0,0,0], linewidth = 0.6, zorder = 3, transform = ramscrs))
    ax.add_patch(mpl.patches.Polygon(patchpoints_between, edgecolor = "black", facecolor = [1,0,0,0], linewidth = 0.6, zorder = 3, transform = ramscrs))
    ax.annotate("A", [-81, 39], horizontalalignment = "center", verticalalignment = "center", fontsize = 10, color = "black", transform = ccrs.PlateCarree())
    # if ax in (ax22, ax23):
    ax.annotate("B", [-80.4, 41.3], horizontalalignment = "center", verticalalignment = "center", fontsize = 10, color = "black", transform = ccrs.PlateCarree())
    ax.annotate("C", [-79.7, 41.6], horizontalalignment = "center", verticalalignment = "center", fontsize = 10, color = "black", transform = ccrs.PlateCarree())
    ax.annotate("D", [-78.9, 41.95], horizontalalignment = "center", verticalalignment = "center", fontsize = 10, color = "black", transform = ccrs.PlateCarree())
    # ax.annotate("E", [-79, 39.9], horizontalalignment = "center", verticalalignment = "center", fontsize = 8, color = "black", transform = ccrs.PlateCarree())
    termp = ax.pcolormesh(control_snow_old["x"], control_snow_old["y"], control_snow_old["Topo"], cmap = modtopocmap, vmin = 0, vmax = 1000, zorder = 0, transform = ramscrs)

#Because of the heavy snow in Region B in CONTROL (leading to dark red in ax21), we need to plot in white there instead of black
# ax21.annotate("B", [-80.4, 41.3], horizontalalignment = "center", verticalalignment = "center", fontsize = 10, color = "white", transform = ccrs.PlateCarree())

    # ax.pcolormesh(afile_control["lon1d"], afile_control["lat1d"], afile_control["PatchArea"][0,:,:].where(afile_control["PatchArea"][0,:,:]==1), color = "Navy", zorder = 0)
ax11.set_title("CONTROL")
ax12.set_title("NLH")
ax13.set_title("CONTROL-NLH")
# fig.suptitle(f"24-Hour Snowfall Ending {(endt).strftime('%d')} Jan - {(endt).strftime('%H%M')} UTC", y = 0.96)
from palettable.lightbartlein.diverging import BrownBlue10_10
from palettable.lightbartlein.sequential import Blues10_10
snwcmap = Blues10_10.get_mpl_colormap()
snwdiffcmap = BrownBlue10_10.get_mpl_colormap()
ax11.pcolormesh(control_snow_old["x"], control_snow_old["y"], control_snow_old["PatchArea"][0,:,:].where(control_snow_old["PatchArea"][0,:,:]>0.01), vmin = 0, vmax = 0.1, cmap = "Blues", shading = "nearest", zorder = 1, transform = ramscrs)
ax12.pcolormesh(control_snow_old["x"], control_snow_old["y"], nolake_snow_old["PatchArea"][0,:,:].where(nolake_snow_old["PatchArea"][0,:,:]>0.01), vmin = 0, vmax = 0.1, cmap = "Blues", shading = "nearest", zorder = 1, transform = ramscrs)
snowmp = ax11.pcolormesh(control_snow_old["x"], control_snow_old["y"], snowdiff_control.where(snowdiff_control>0.1), shading = "nearest", cmap = snwcmap, vmin = 0, vmax = 30, zorder = 2, transform = ramscrs)
ax12.pcolormesh(control_snow_old["x"], control_snow_old["y"], snowdiff_nolake.where(snowdiff_nolake>0.1), shading = "nearest", cmap = snwcmap, vmin = 0, vmax = 30, zorder = 2, transform = ramscrs)
diffmp = ax13.pcolormesh(control_snow_old["x"], control_snow_old["y"], snowdiff_control-snowdiff_nolake, shading = "nearest", cmap = snwdiffcmap, vmin = -15, vmax = 15, zorder = 2, transform = ramscrs)
# # # ax1.contour(afile_control["lon1d"], afile_control["lat1d"], afile_control["PatchArea"][1,:,:], levels = [0.999], colors = "white", linestyles = "-", linewidths = 0.7)
# # # ax1.legend(loc = "lower left", fontsize = 20, handles = [fakecontour])
# # # ax2.contour(afile_control["lon1d"], afile_control["lat1d"], afile_nolake["PatchArea"][1,:,:], levels = [0.999], colors = "white", linestyles = "-", linewidths = 0.7)
# # # ax2.legend(loc = "lower left", fontsize = 20, handles = [fakecontour])

ax21.set_title("CONTROL")
ax22.set_title("NLH")
ax23.set_title("CONTROL-NLH")
ax21.pcolormesh(control_snow_old["x"][200:750], control_snow_old["y"][150:650], control_snow_old["PatchArea"][0,150:650,200:750].where(control_snow_old["PatchArea"][0,150:650,200:750]>0.01), vmin = 0, vmax = 0.1, cmap = "Blues", shading = "nearest", zorder = 1, transform = ramscrs)
ax22.pcolormesh(control_snow_old["x"][200:750], control_snow_old["y"][150:650], nolake_snow_old["PatchArea"][0,150:650,200:750].where(nolake_snow_old["PatchArea"][0,150:650,200:750]>0.01), vmin = 0, vmax = 0.1, cmap = "Blues", shading = "nearest", zorder = 1, transform = ramscrs)
ax21.pcolormesh(control_snow_old["x"][200:750], control_snow_old["y"][150:650], snowdiff_control[150:650,200:750].where(snowdiff_control[150:650,200:750]>0.1), shading = "nearest", cmap = snwcmap, vmin = 0, vmax = 30, zorder = 2, transform = ramscrs)
ax22.pcolormesh(control_snow_old["x"][200:750], control_snow_old["y"][150:650], snowdiff_nolake[150:650,200:750].where(snowdiff_nolake[150:650,200:750]>0.1), shading = "nearest", cmap = snwcmap, vmin = 0, vmax = 30, zorder = 2, transform = ramscrs)
ax23.pcolormesh(control_snow_old["x"][200:750], control_snow_old["y"][150:650], snowdiff_control[150:650,200:750]-snowdiff_nolake[150:650,200:750], shading = "nearest", cmap = snwdiffcmap, vmin = -15, vmax = 15, zorder = 2, transform = ramscrs)
fig.get_layout_engine().set(w_pad = 6/72, h_pad = 3/72, wspace = 0.05, hspace = 0)

snowcbar = fig.colorbar(snowmp, ax = [ax11,ax21], orientation = "horizontal", fraction = 0.05, extend = "max", pad = 0.05); snowcbar.set_label("Total Snowfall (mm Liq. Eq.)")
tercbar = fig.colorbar(termp, ax = [ax12,ax22], orientation = "horizontal", fraction = 0.05, extend = "max", pad = 0.05); tercbar.set_label("Terrain Height (m)")
snwdiffcbar = fig.colorbar(diffmp, ax = [ax13,ax23], orientation = "horizontal", fraction = 0.05, extend = "both", pad = 0.05); snwdiffcbar.set_label("Diff. in Total Snowfall")
fig.savefig(f"../PaperFigs/SnowTotCompcarto_{endt.strftime('%d%H%M')}z.png")
plt.close();
del ax11; del ax12; del ax13
del ax21; del ax22; del ax23
del fig; del snowcbar; del tercbar; del snwdiffcbar
del snowmp; del diffmp; del termp

In [ ]:
snowdiff_control_old.close(); del snowdiff_control_old
control_snow_old.close(); del control_snow_old
nolake_snow_old.close(); del nolake_snow_old
control_snow_new.close(); del control_snow_new
nolake_snow_new.close(); del nolake_snow_new
snowdiff_nolake.close(); del snowdiff_nolake

### Calculating/Plotting changes in snowfall between CONTROL, VARTEMP, and MORESNOW

In [ ]:
#Note that there we're using the raw RAMS output rather than post-processed files. This is because we're only examining a scalar surface field (snowfall) which is unchanged in the post-processing routine.
controlrawpath = input("Enter the directory containing the RAMS output files for the CONTROL simulation: ").rstrip("/ ")
if not os.path.exists(controlpath):
    raise Exception("CONTROL analysis file directory not found!")
control_snow_old = xr.open_dataset(f"{controlrawpath}/a-A-2010-01-02-060000-g1.h5")
control_snow_new = xr.open_dataset(f"{controlrawpath}/a-A-2010-01-03-060000-g1.h5")
vartemprawpath = input("Enter the directory containing the RAMS output files for the VARTEMP simulation: ").rstrip("/ ")
if not os.path.exists(vartemppath):
    raise Exception("VARTEMP analysis file directory not found!")
vartemp_snow_old = xr.open_dataset(f"{vartemprawpath}/a-A-2010-01-02-060000-g1.h5")
vartemp_snow_new = xr.open_dataset(f"{vartemprawpath}/a-A-2010-01-03-060000-g1.h5")
moresnowrawpath = input("Enter the directory containing the RAMS output files for the MORESNOW simulation: ").rstrip("/ ")
if not os.path.exists(moresnowpath):
    raise Exception("MORESNOW analysis file directory not found!")
moresnow_snow_old = xr.open_dataset(f"{moresnowrawpath}/a-A-2010-01-02-060000-g1.h5")
moresnow_snow_new = xr.open_dataset(f"{moresnowrawpath}/a-A-2010-01-03-060000-g1.h5")

In [ ]:
snowdiff_control_vals = control_snow_new["ACCPS"]+control_snow_new["ACCPA"]-control_snow_old["ACCPS"]-control_snow_old["ACCPA"]
snowdiff_vartemp_vals = vartemp_snow_new["ACCPS"]+vartemp_snow_new["ACCPA"]-vartemp_snow_old["ACCPS"]-vartemp_snow_old["ACCPA"]
snowdiff_moresnow_vals = moresnow_snow_new["ACCPS"]+moresnow_snow_new["ACCPA"]-moresnow_snow_old["ACCPS"]-moresnow_snow_old["ACCPA"]
snowdiff_control_vals = snowdiff_control_vals[1:-1,1:-1].values
snowdiff_vartemp_vals = snowdiff_vartemp_vals[1:-1,1:-1].values
snowdiff_moresnow_vals = snowdiff_moresnow_vals[1:-1,1:-1].values
#This [1:-1,1:-1] business is because the raw RAMS output includes lateral boundary points which are not actually used in computation

In [ ]:
snowdiff_control = xr.DataArray(snowdiff_control_vals, coords = {"y": controlgrid["y"], "x": controlgrid["x"]}, dims = ["y", "x"])
snowdiff_vartemp = xr.DataArray(snowdiff_vartemp_vals, coords = {"y": controlgrid["y"], "x": controlgrid["x"]}, dims = ["y", "x"])
snowdiff_moresnow = xr.DataArray(snowdiff_moresnow_vals, coords = {"y": controlgrid["y"], "x": controlgrid["x"]}, dims = ["y", "x"])

In [ ]:
ix0 = 400; iy0 = 525 #Northwest side of the "downwind of lake erie" box, at the center of the downwind shore
x0 = controlgrid["x"].isel(x = ix0); y0 = controlgrid["y"].isel(y = iy0); dxy = np.diff(controlgrid["x"])[0]
width = 400000; length = 305000 #m, dimensions of the box
tarray = np.arange(0, length+1, dxy); narray = np.arange(-width/2, width/2+1, dxy) #Coordinates of the box in tangent (long-axis) and normal (short-axis) directions. Normal is negative and positive (northeast and southwest), while tangent is only positive (to the right)
tmesh, nmesh = np.meshgrid(tarray, narray)
angle = np.pi/180*(-60) #Angle of the long axis of the box, with respect to the horizontal
xbox = x0.values+np.cos(angle)*tmesh-np.sin(angle)*nmesh
ybox = y0.values+np.sin(angle)*tmesh+np.cos(angle)*nmesh
snowbox = xr.DataArray(coords = {"y": (("normal", "tangent"), ybox), "x": (("normal", "tangent"), xbox), "normal": narray, "tangent": tarray}, dims = ["normal", "tangent"])

downwindsnow_control = snowdiff_control.interp(x = snowbox["x"], y = snowbox["y"])
downwindsnow_vartemp = snowdiff_vartemp.interp(x = snowbox["x"], y = snowbox["y"])
downwindsnow_moresnow = snowdiff_moresnow.interp(x = snowbox["x"], y = snowbox["y"])

print(f"Average liquid equivalent of snowfall (mm) downwind of Lake Erie (Region A) in the CONTROl simulation \n{downwindsnow_control.mean():.2f} \nAverage liquid equivalent of snowfall downwind of Lake Erie (Region A) in the VARTEMP simulation \n{downwindsnow_vartemp.mean():.2f} \nAverage liquid equivalent of snowfall downwind of Lake Erie (Region A) in the MORESNOW simulation \n{downwindsnow_moresnow.mean():.2f} \nRelative change in average snowfall downwind of Lake Erie (Region A) between CONTROL and VARTEMP simulations \n{100*(downwindsnow_control.mean()-downwindsnow_vartemp.mean())/downwindsnow_control.mean():.2f}% \nRelative change in average snowfall downwind of Lake Erie (Region A) between CONTROL and MORESNOW simulations \n{100*(downwindsnow_control.mean()-downwindsnow_moresnow.mean())/downwindsnow_control.mean():.2f}% \nAverage CONTROL simulation snowfall downwind of Lake Erie (Region A) is {100*downwindsnow_control.mean()/downwindsnow_vartemp.mean():.2f}% of average VARTEMP simulation snowfall \nAverage CONTROL simulation snowfall downwind of Lake Erie (Region A) is {100*downwindsnow_control.mean()/downwindsnow_moresnow.mean():.2f}% of average MORESNOW simulation snowfall")
print("\n\n")


patchpoints = np.zeros((4, 2))
patchpoints[:,0] = [downwindsnow_control["x"].values[0,0], downwindsnow_control["x"].values[0, -1], downwindsnow_control["x"].values[-1, -1], downwindsnow_control["x"].values[-1, 0]]
patchpoints[:,1] = [downwindsnow_control["y"].values[0,0], downwindsnow_control["y"].values[0, -1], downwindsnow_control["y"].values[-1, -1], downwindsnow_control["y"].values[-1, 0]]

print(f"Average liquid equivalent of snowfall (mm) in the main band (Region B) in the CONTROl simulation \n{downwindsnow_control[210:250, 0:100].mean():.2f} \nAverage liquid equivalent of snowfall in the main band (Region B) in the VARTEMP simulation \n{downwindsnow_vartemp[210:250, 0:100].mean():.2f} \nAverage liquid equivalent of snowfall in the main band (Region B) in the MORESNOW simulation \n{downwindsnow_moresnow[210:250, 0:100].mean():.2f} \nRelative change in average snowfall in the main band (Region B) between CONTROL and VARTEMP simulations \n{100*(downwindsnow_control[210:250, 0:100].mean()-downwindsnow_vartemp[210:250, 0:100].mean())/downwindsnow_control[210:250, 0:100].mean():.2f}% \nRelative change in average snowfall in the main band (Region B) between CONTROL and MORESNOW simulations \n{100*(downwindsnow_control[210:250, 0:100].mean()-downwindsnow_moresnow[210:250, 0:100].mean())/downwindsnow_control[210:250, 0:100].mean():.2f}% \nAverage CONTROL simulation snowfall in the main band (Region B) is {100*downwindsnow_control[210:250, 0:100].mean()/downwindsnow_vartemp[210:250, 0:100].mean():.2f}% of average VARTEMP simulation snowfall \nAverage CONTROL simulation snowfall in the main band (Region B) is {100*downwindsnow_control[210:250, 0:100].mean()/downwindsnow_moresnow[210:250, 0:100].mean():.2f}% of average MORESNOW simulation snowfall")
print("\n\n")
#Change in snowfall in main band
#Points 210:250, 0:100 form a rectangle roughly bounding the main snow band over Lake Erie in the CONTROL simulation
patchpointssub = np.zeros((4, 2)) #These are the points for box B in the figure below, outlining the main band
patchpointssub[:,0] = [downwindsnow_control["x"].values[210,0], downwindsnow_control["x"].values[210, 100], downwindsnow_control["x"].values[250, 100], downwindsnow_control["x"].values[250, 0]]
patchpointssub[:,1] = [downwindsnow_control["y"].values[210,0], downwindsnow_control["y"].values[210, 100], downwindsnow_control["y"].values[250, 100], downwindsnow_control["y"].values[250, 0]]
# print(downwindsnow_control[210:250, 0:100].mean(), downwindsnow_nolake[210:250, 0:100].mean(), downwindsnow_control[210:250, 0:100].sum()/downwindsnow_nolake[210:250, 0:100].sum())

print(f"Average liquid equivalent of snowfall (mm) downwind of NE Lake Erie (Region D) in the CONTROl simulation \n{downwindsnow_control[350:400, 0:100].mean():.2f} \nAverage liquid equivalent of snowfall downwind of NE Lake Erie (Region D) in the VARTEMP simulation \n{downwindsnow_vartemp[350:400, 0:100].mean():.2f} \nAverage liquid equivalent of snowfall downwind of NE Lake Erie (Region D) in the MORESNOW simulation \n{downwindsnow_moresnow[350:400, 0:100].mean():.2f} \nRelative change in average snowfall downwind of NE Lake Erie (Region D) between CONTROL and VARTEMP simulations \n{100*(downwindsnow_control[350:400, 0:100].mean()-downwindsnow_vartemp[350:400, 0:100].mean())/downwindsnow_control[350:400, 0:100].mean():.2f}% \nRelative change in average snowfall downwind of NE Lake Erie (Region D) between CONTROL and MORESNOW simulations \n{100*(downwindsnow_control[350:400, 0:100].mean()-downwindsnow_moresnow[350:400, 0:100].mean())/downwindsnow_control[350:400, 0:100].mean():.2f}% \nAverage CONTROL simulation snowfall downwind of NE Lake Erie (Region D) is {100*downwindsnow_control[350:400, 0:100].mean()/downwindsnow_vartemp[350:400, 0:100].mean():.2f}% of average VARTEMP simulation snowfall \nAverage CONTROL simulation snowfall downwind of NE Lake Erie (Region D) is {100*downwindsnow_control[350:400, 0:100].mean()/downwindsnow_moresnow[350:400, 0:100].mean():.2f}% of average MORESNOW simulation snowfall") #Change in snowfall in Region D
#Points 350:400, 0:100 deliniate Region D, where there's enhanced snowfall downwind of NE Lake Erie in CONTROL and NLH
patchpointsne = np.zeros((4,2)) #Points bounding the band in northeast Lake Erie (box D)
patchpointsne[:,0] = [downwindsnow_control["x"].values[350,0], downwindsnow_control["x"].values[350, 100], downwindsnow_control["x"].values[400, 100], downwindsnow_control["x"].values[400, 0]]
patchpointsne[:,1] = [downwindsnow_control["y"].values[350,0], downwindsnow_control["y"].values[350, 100], downwindsnow_control["y"].values[400, 100], downwindsnow_control["y"].values[400, 0]]


patchpoints_between = np.zeros((4, 2)) #Points bounding area downwind of Lake Erie between box B and box D (box C)
patchpoints_between[:,0] = [downwindsnow_control["x"].values[250,0], downwindsnow_control["x"].values[250, 100], downwindsnow_control["x"].values[350, 100], downwindsnow_control["x"].values[350, 0]]
patchpoints_between[:,1] = [downwindsnow_control["y"].values[250,0], downwindsnow_control["y"].values[250, 100], downwindsnow_control["y"].values[350, 100], downwindsnow_control["y"].values[350, 0]]

In [ ]:
ramscrs = ccrs.Stereographic(central_longitude = -80, central_latitude = 42)
fig, (ax11, ax12, ax13) = plt.subplots(1, 3, figsize = (8, 3), dpi = 200, layout = "compressed", subplot_kw = {"projection": ramscrs})
startt = datetime(2010, 1, 2, 6, 0, 0); print(startt)
endt = datetime(2010, 1, 3, 6, 0, 0); print(endt)
modtopocmap = truncate_colormap(mpl.colormaps.get_cmap("gist_earth"), minval = 0.5, maxval = 1, n = 128)
axlabelsbot = ["(a)", "(b)", "(c)"]
from palettable.lightbartlein.diverging import BrownBlue10_10
snwdiffcmap = BrownBlue10_10.get_mpl_colormap()
for i, ax in enumerate((ax11, ax12, ax13)):
    ax.set_extent((controlgrid["x"][200], controlgrid["x"][750], controlgrid["y"][150], controlgrid["y"][650]), crs = ramscrs)
    # ax.annotate(axlabels[i], (0.04, 0.94), xycoords = "axes fraction", color = "black", fontsize = 6.5, horizontalalignment = "center", bbox = {"edgecolor": "black", "linewidth": 1, "facecolor": "white"}, zorder = 2)
    ax.add_feature(cfeature.NaturalEarthFeature('physical', 'coastline', '50m', edgecolor = 'darkgrey', facecolor = "none", linewidth = 0.4, zorder = 3))
    # ax.add_feature(cfeature.NaturalEarthFeature('cultural', 'admin_0_boundary_lines_land', '50m', edgecolor = "black", facecolor = "none", linewidth = 0.4, zorder = 3))
    # ax.add_feature(cfeature.NaturalEarthFeature('physical', 'lakes', '50m',
    #                                             edgecolor='navy',
    #                                             facecolor="dodgerblue", linewidth = 0.2), zorder = 1)
    # ax.add_feature(cfeature.NaturalEarthFeature(category='cultural', name='admin_1_states_provinces_lines', facecolor='none', scale = "50m", edgecolor='darkgrey', zorder = 2))
    ax.add_feature(cfeature.NaturalEarthFeature(category="cultural", name = "admin_1_states_provinces_lakes", linewidth = 0.4, facecolor = "none", scale = "50m", edgecolor = "black", zorder = 3))
    
    left, width = 0, 0.1
    bottom, height = 0.90, 0.1
    right = left + width
    top = bottom + height
    p = plt.Rectangle((left, bottom), width, height, fill=True, zorder = 3, edgecolor = "black", linewidth = 0.2, facecolor = "white")
    p.set_transform(ax.transAxes)
    p.set_clip_on(False)
    ax.add_patch(p)
    ax.text(left+0.5*width, bottom+0.5*height, axlabelsbot[i], fontsize = 10, transform = ax.transAxes, horizontalalignment = "center", verticalalignment = "center")
    ax.add_patch(mpl.patches.Polygon(patchpoints, edgecolor = "black", facecolor = [1, 0, 0, 0], linewidth = 0.6, zorder = 3, transform = ramscrs))
    ax.add_patch(mpl.patches.Polygon(patchpointssub, edgecolor = "black", facecolor = [1,0,0,0], linewidth = 0.6, zorder = 3, transform = ramscrs))
    ax.add_patch(mpl.patches.Polygon(patchpointsne, edgecolor = "black", facecolor = [1,0,0,0], linewidth = 0.6, zorder = 3, transform = ramscrs))
    ax.add_patch(mpl.patches.Polygon(patchpoints_between, edgecolor = "black", facecolor = [1,0,0,0], linewidth = 0.6, zorder = 3, transform = ramscrs))
    ax.annotate("A", [-81, 39], horizontalalignment = "center", verticalalignment = "center", fontsize = 10, color = "black", transform = ccrs.PlateCarree())
    ax.annotate("B", [-80.4, 41.3], horizontalalignment = "center", verticalalignment = "center", fontsize = 10, color = "black", transform = ccrs.PlateCarree())
    ax.annotate("C", [-79.7, 41.6], horizontalalignment = "center", verticalalignment = "center", fontsize = 10, color = "black", transform = ccrs.PlateCarree())
    ax.annotate("D", [-78.9, 41.95], horizontalalignment = "center", verticalalignment = "center", fontsize = 10, color = "black", transform = ccrs.PlateCarree())
gl = ax13.gridlines(crs = ccrs.PlateCarree(), draw_labels = {"bottom": "x", "left": "y"}, x_inline = False, y_inline = False, dms = True)
gl.xlocator = mticker.FixedLocator([-82, -80, -78])
gl.ylocator = mticker.FixedLocator([38, 39, 40, 41, 42])
gl.bottom_labels = True
gl.left_labels   = False
gl.xlines = False
gl.ylines = False
gl.top_labels    = False
gl.right_labels  = True
gl.xlabel_style = {'size': 8, 'color': 'black', 'rotation': 0, "horizontalalignment": "right"}
gl.ylabel_style = {'size': 8, 'color': 'black', 'rotation': 0, "horizontalalignment": "right"}
termp = ax11.pcolormesh(controlgrid["x"], controlgrid["y"], controlgrid["Topo"], cmap = modtopocmap, vmin = 0, vmax = 1000, zorder = 0, transform = ramscrs)
ax12.pcolormesh(controlgrid["x"], controlgrid["y"], controlgrid["Topo"], cmap = modtopocmap, vmin = 0, vmax = 1000, zorder = 0, transform = ramscrs)
ax13.pcolormesh(controlgrid["x"], controlgrid["y"], controlgrid["Topo"], cmap = modtopocmap, vmin = 0, vmax = 1000, zorder = 0, transform = ramscrs)

#Because of the heavy snow in Region B in CONTROL (leading to dark red in ax21), we need to plot in white there instead of black
ax11.set_title("CONTROL")
ax12.set_title("VARTEMP")
ax13.set_title("MORESNOW")
from palettable.lightbartlein.sequential import Blues10_10
snwcmap = Blues10_10.get_mpl_colormap()
ax11.pcolormesh(controlgrid["x"][200:750], controlgrid["y"][150:650], controlgrid["PatchArea"][0,150:650,200:750].where(controlgrid["PatchArea"][0,150:650,200:750]>0.01), vmin = 0, vmax = 0.1, cmap = "Blues", shading = "nearest", zorder = 1, transform = ramscrs)
ax12.pcolormesh(controlgrid["x"][200:750], controlgrid["y"][150:650], controlgrid["PatchArea"][0,150:650,200:750].where(controlgrid["PatchArea"][0,150:650,200:750]>0.01), vmin = 0, vmax = 0.1, cmap = "Blues", shading = "nearest", zorder = 1, transform = ramscrs)
ax13.pcolormesh(controlgrid["x"][200:750], controlgrid["y"][150:650], controlgrid["PatchArea"][0,150:650,200:750].where(controlgrid["PatchArea"][0,150:650,200:750]>0.01), vmin = 0, vmax = 0.1, cmap = "Blues", shading = "nearest", zorder = 1, transform = ramscrs)
snowmp = ax11.pcolormesh(controlgrid["x"][200:750], controlgrid["y"][150:650], snowdiff_control[150:650,200:750].where(snowdiff_control[150:650,200:750]>0.1), shading = "nearest", cmap = snwcmap, vmin = 0, vmax = 30, zorder = 2, transform = ramscrs)
ax12.pcolormesh(controlgrid["x"][200:750], controlgrid["y"][150:650], snowdiff_vartemp[150:650,200:750].where(snowdiff_vartemp[150:650,200:750]>0.1), shading = "nearest", cmap = snwcmap, vmin = 0, vmax = 30, zorder = 2, transform = ramscrs)
ax13.pcolormesh(controlgrid["x"][200:750], controlgrid["y"][150:650], snowdiff_moresnow[150:650,200:750].where(snowdiff_moresnow[150:650,200:750]>0.1), shading = "nearest", cmap = snwcmap, vmin = 0, vmax = 30, zorder = 2, transform = ramscrs)
fig.get_layout_engine().set(w_pad = 6/72, h_pad = 3/72, wspace = 0.05, hspace = 0)

snowcbar = fig.colorbar(snowmp, ax = [ax11, ax12, ax13], orientation = "horizontal", fraction = 0.1, extend = "max", pad = 0.05, aspect = 45); snowcbar.set_label("Total Snowfall (mm Liq. Eq.)", fontsize = 10)
tercbar = fig.colorbar(termp, ax = [ax11, ax12, ax13], orientation = "vertical", location = "left", fraction = 0.1, extend = "max", pad = 0.02); tercbar.set_label("Terrain Height (m)", fontsize = 10, labelpad = 0.02)
fig.savefig(f"../PaperFigs/SnowTotSensitivity_SameColorscale{endt.strftime('%d%H%M')}z.png")
# plt.close();
del ax11; del ax12; del ax13
del fig; del snowcbar; del tercbar;
del snowmp; del termp

In [ ]:
control_snow_old.close(); del control_snow_old
control_snow_new.close(); del control_snow_new
vartemp_snow_old.close(); del vartemp_snow_old
vartemp_snow_new.close(); del vartemp_snow_new
moresnow_snow_old.close(); del moresnow_snow_old
moresnow_snow_new.close(); del moresnow_snow_new
del snowdiff_control_vals
del snowdiff_vartemp_vals
del snowdiff_moresnow_vals
snowdiff_control.close(); del snowdiff_control
snowdiff_vartemp.close(); del snowdiff_vartemp
snowdiff_moresnow.close(); del snowdiff_moresnow
downwindsnow_control.close(); del downwindsnow_control
downwindsnow_vartemp.close(); del downwindsnow_vartemp
downwindsnow_moresnow.close(); del downwindsnow_moresnow

## Read in w data from 24-hour period (this takes some time)

In [ ]:
ftlist = date_range("2010-01-02-060000", "2010-01-03-060000", freq = "20min")
ftstrings = [i.strftime("%Y-%m-%d-%H%M%S") for i in ftlist]

controlws = xr.open_mfdataset([f"{controlpath}/mvars-cart-{i}-g1.nc" for i in ftstrings], preprocess = lambda x: x[["w", "SnowPrecipRate", "AggPrecipRate"]], combine = "nested", concat_dim = "time", engine = "h5netcdf")
nolakews = xr.open_mfdataset([f"{nlhpath}/mvars-cart-{i}-g1.nc" for i in ftstrings], preprocess = lambda x: x[["w", "SnowPrecipRate", "AggPrecipRate"]], combine = "nested", concat_dim = "time", engine = "h5netcdf")
controlws["time"] = ftlist
nolakews["time"] = ftlist

In [ ]:
walt = 700
meanw_control = controlws["w"].sel(z = 700, method = "nearest").mean(dim = "time")
meanw_nolake = nolakews["w"].sel(z =700, method = "nearest").mean(dim = "time")

### Plot composite plan view of w (this can take several minutes)

In [ ]:
ramscrs = ccrs.Stereographic(central_longitude = -80, central_latitude = 42)
fig, ((ax1, ax2), (ax3,ax4)) = plt.subplots(2, 2, figsize = (5, 6), dpi = 200, layout = "constrained", subplot_kw = {"projection": ramscrs})
axlabels = ["(a)", "(b)", "(c)", "(d)"]
for i,ax in enumerate((ax1, ax2, ax3, ax4)):
    ax = add_cartofeatures_zoomdomain(ax)
    left, width = 0, 0.1
    bottom, height = 0.90, 0.1
    right = left + width
    top = bottom + height
    p = plt.Rectangle((left, bottom), width, height, fill=True, zorder = 3, edgecolor = "black", linewidth = 0.2, facecolor = "white")
    p.set_transform(ax.transAxes)
    p.set_clip_on(False)
    ax.add_patch(p)
    ax.text(left+0.5*width, bottom+0.5*height, axlabels[i], fontsize = 10, transform = ax.transAxes, horizontalalignment = "center", verticalalignment = "center")
    ax.plot([controlgrid["x"][350], controlgrid["x"][350]], [controlgrid["y"][480], controlgrid["y"][680]], color = "black", linestyle = "--", linewidth = 1, zorder = 4, transform = ramscrs)
    ax.plot([controlgrid["x"][550], controlgrid["x"][550]], [controlgrid["y"][480], controlgrid["y"][680]], color = "black", linestyle = "--", linewidth = 1, zorder = 2)
    ax.plot([controlgrid["x"][350], controlgrid["x"][550]], [controlgrid["y"][480], controlgrid["y"][480]], color = "black", linestyle = "--", linewidth = 1, zorder = 2)
    ax.plot([controlgrid["x"][350], controlgrid["x"][550]], [controlgrid["y"][680], controlgrid["y"][680]], color = "black", linestyle = "--", linewidth = 1, zorder = 2)
for i, geoax in enumerate((ax3, ax4)):
    geoax.set_extent((controlgrid["x"][350], controlgrid["x"][550], controlgrid["y"][480], controlgrid["y"][680]), crs = ramscrs)
    # ax.annotate(axlabels[i], (0.04, 0.94), xycoords = "axes fraction", color = "black", fontsize = 6.5, horizontalalignment = "center", bbox = {"edgecolor": "black", "linewidth": 1, "facecolor": "white"}, zorder = 2)
    geoax.add_feature(cfeature.NaturalEarthFeature('physical', 'coastline', '50m', edgecolor = 'darkgrey', facecolor = "none", linewidth = 0.4, zorder = 3))
    # ax.add_feature(cfeature.NaturalEarthFeature('cultural', 'admin_0_boundary_lines_land', '50m', edgecolor = "black", facecolor = "none", linewidth = 0.4, zorder = 3))
    # ax.add_feature(cfeature.NaturalEarthFeature('physical', 'lakes', '50m',
    #                                             edgecolor='navy',
    #                                             facecolor="dodgerblue", linewidth = 0.2), zorder = 1)
    # ax.add_feature(cfeature.NaturalEarthFeature(category='cultural', name='admin_1_states_provinces_lines', facecolor='none', scale = "50m", edgecolor='darkgrey', zorder = 2))
    geoax.add_feature(cfeature.NaturalEarthFeature(category="cultural", name = "admin_1_states_provinces_lakes", linewidth = 0.4, facecolor = "none", scale = "50m", edgecolor = "black", zorder = 3))
    
    gl = geoax.gridlines(crs = ccrs.PlateCarree(), draw_labels = {"bottom": "x", "left": "y"}, x_inline = False, y_inline = False, dms = True)
    gl.xlocator = mticker.FixedLocator([-81, -80, -79])
    gl.ylocator = mticker.FixedLocator([40, 41, 42, 43])
    gl.bottom_labels = True
    gl.left_labels   = True
    gl.xlines = False
    gl.ylines = False
    gl.top_labels    = False
    gl.right_labels  = False
    gl.xlabel_style = {'size': 8, 'color': 'black', 'rotation': 0, "horizontalalignment": "right"}
    gl.ylabel_style = {'size': 8, 'color': 'black', 'rotation': 0, "horizontalalignment": "right"}
# ax1.set_title("CONTROL", fontfamily = "Liberation Serif", y = 1.02)
# ax2.set_title("NLH", fontfamily = "Liberation Serif", y = 1.02)
# ax1.contour(controlgrid["x"].isel(x = slice(50, 650)), controlgrid["y"].isel(y = slice(450, 1050)), controlgrid["PatchArea"].isel(patch = 1, x = slice(50, 650), y = slice(450, 1050)), levels = [0.99], colors = "black", linewidths = 0.5, zorder = 1)
# ax2.contour(controlgrid["x"].isel(x = slice(50, 650)), controlgrid["y"].isel(y = slice(450, 1050)), nlhgrid["PatchArea"].isel(patch = 1, x = slice(50, 650), y = slice(450, 1050)), levels = [0.99], colors = "black", linewidths = 0.5, zorder = 1)
walt = 700 #m AMSL, altitude at which to evalute w
fig.suptitle(f"Time-Averaged 700m AMSL Vertical Velocity", fontsize = 13)
from matplotlib.lines import Line2D
ax1.plot([350, 350], [480, 680])
# fakecontour = Line2D([], [], color = "black", linewidth = 1.5, label = "Water Bodies")
wmp = ax1.pcolormesh(controlgrid["x"].isel(x = slice(50, 650)), controlgrid["y"].isel(y = slice(450, 1050)), meanw_control.isel(x = slice(50, 650), y = slice(450, 1050)), cmap = "RdBu_r", norm = MidpointNormalize(vmin = -0.25, vmax = 0.5, vcenter = 0), zorder = 0)
ax2.pcolormesh(controlgrid["x"].isel(x = slice(50, 650)), controlgrid["y"].isel(y = slice(450, 1050)), meanw_nolake.isel(x = slice(50, 650), y = slice(450, 1050)), cmap = "RdBu_r", norm = MidpointNormalize(vmin = -0.25, vmax = 0.5, vcenter = 0), zorder = 0)
ax3.pcolormesh(controlgrid["x"].isel(x = slice(350, 550)), controlgrid["y"].isel(y = slice(480, 680)), meanw_control.isel(x = slice(350, 550), y = slice(480, 680)), cmap = "RdBu_r", norm = MidpointNormalize(vmin = -0.25, vmax = 0.5, vcenter = 0), zorder = 0)
ax4.pcolormesh(controlgrid["x"].isel(x = slice(350, 550)), controlgrid["y"].isel(y = slice(480, 680)), meanw_nolake.isel(x = slice(350, 550), y = slice(480, 680)), cmap = "RdBu_r", norm = MidpointNormalize(vmin = -0.25, vmax = 0.5, vcenter = 0), zorder = 0)
# ax1.pcolormesh(controlgrid["x"].isel(x = slice(50, 650)), controlgrid["y"].isel(y = slice(450, 1050)), np.zeros((600, 600)), cmap = "BrBG", vmin = -0.5, vmax = 0.5)
# ax2.pcolormesh(controlgrid["x"].isel(x = slice(50, 650)), controlgrid["y"].isel(y = slice(450, 1050)), np.zeros((600, 600)), cmap = "BrBG", vmin = -0.5, vmax = 0.5)
# ax1.legend(loc = "lower left", fontsize = 20, handles = [fakecontour], labelcolor = "black")
# ax2.legend(loc = "lower left", fontsize = 20, handles = [fakecontour], labelcolor = "black")
ax1.set_title("CONTROL", fontsize = 12)
ax2.set_title("NLH", fontsize = 12)
cbar = fig.colorbar(wmp, ax = [ax1, ax2, ax3, ax4], orientation = "horizontal", fraction = 0.05, extend = "both"); cbar.set_label(f"{walt}m AMSL Vertical Velocity ($\mathrm{{m \ s^{{-1}}}}$)", fontsize = 10)
cbar.ax.tick_params(labelsize = 8)
fig.savefig(f"../PaperFigs/wplan_composite_carto_4panel.png")
# plt.close(); del cbar; del ax1; del ax2; del wmp; del fig

In [ ]:
if not os.path.exists("lakebox.nc"):
    ix0 = 380; iy0 = 615 #Northwest side of the "downwind of lake erie" box, at the center of the downwind shore
    x0 = controlgrid["x"].isel(x = ix0); y0 = controlgrid["y"].isel(y = iy0); dxy = np.diff(controlgrid["x"])[0]
    width = 380000; length = 105000 #m, dimensions of the box
    tarray = np.arange(0, length+1, dxy); narray = np.arange(-width/2, width/2+1, dxy) #Coordinates of the box in tangent (long-axis) and normal (short-axis) directions. Normal is negative and positive (northeast and southwest), while tangent is only positive (to the right)
    tmesh, nmesh = np.meshgrid(tarray, narray)
    print(tmesh)
    print(nmesh)
    angle = np.pi/180*(-70) #Angle of the long axis of the box, with respect to the horizontal
    xbox = x0.values+np.cos(angle)*tmesh-np.sin(angle)*nmesh
    ybox = y0.values+np.sin(angle)*tmesh+np.cos(angle)*nmesh
    lakebox = xr.DataArray(coords = {"y": (("normal", "tangent"), ybox), "x": (("normal", "tangent"), xbox), "normal": narray, "tangent": tarray}, dims = ["normal", "tangent"])
    lakebox["lat"] = controlgrid["lat2d"].interp(x = lakebox["x"], y = lakebox["y"])
    lakebox["lon"] = controlgrid["lon2d"].interp(x = lakebox["x"], y = lakebox["y"])
    lakebox["lakemask"] = xr.DataArray(np.where(controlgrid["PatchArea"][0,:,:].interp(x = lakebox["x"], y = lakebox["y"])>0.99, 1, np.nan), coords = lakebox.coords, dims = lakebox.dims)
    lakebox.to_netcdf("lakebox.nc")

## Skew-T comparing thermodynamics in CONTROL and NLH simulations

In [ ]:
afile_control = xr.open_dataset(f"{controlpath}/mvars-cart-2010-01-02-230000-g1.nc")
afile_nolake = xr.open_dataset(f"{nlhpath}/mvars-cart-2010-01-02-230000-g1.nc")

controlthermo = afile_control[["Temperature", "Theta", "Pressure", "Dewpoint", "VaporMix", "VaporPressure"]]
nolakethermo = afile_nolake[["Temperature", "Theta", "Pressure", "Dewpoint", "VaporMix", "VaporPressure"]]
lakebox = xr.open_dataset("lakebox.nc")
afile_control.close(); del afile_control
afile_nolake.close(); del afile_nolake
eriethermo_control = (controlthermo.interp(x = lakebox["x"], y = lakebox["y"])*lakebox["lakemask"]).mean(dim = ("tangent", "normal")) #This averages all thermodynamic fields for the CONTROL sim horizontally over Lake Erie ONLY (lakemask filters out land)
eriethermo_nolake = (nolakethermo.interp(x = lakebox["x"], y = lakebox["y"])*lakebox["lakemask"]).mean(dim = ("tangent", "normal")) #ditto, for NLH simulation

In [ ]:
import metpy.calc as mcalc
from metpy.plots import SkewT
from metpy.units import units
# usestyle("paperplots.mplstyle")
from matplotlib import rcParams
# rcParams["figure.titlesize"] = 10
# rcParams["axes.titlesize"] = 8
# rcParams["axes.labelsize"] = 8
# rcParams["xtick.labelsize"] = 6.5
# rcParams["ytick.labelsize"] = 6.5
# rcParams["axes.linewidth"] = 0.2
tfig = plt.figure(figsize = (5.5, 4), dpi = 200)
# tfig.suptitle(f"Sounding Launch at time {sound05['time'][0].values}")
testskew = SkewT(tfig, rotation = 45)
testskew.ax.set_ylim(1050, 600)
testskew.ax.set_xlim(-20, 2)
testskew.ax.tick_params(labelsize = 8)
# testskew.ax.set_facecolor("whitesmoke")
# testskew.ax.set_xticks(color = "black")
# testskew.ax.set_edgecolor("black")
# testskew.ax.set_labelcolor("black")
contemp, = testskew.plot(eriethermo_control["Pressure"]*units.hPa, eriethermo_control["Temperature"]*units.K-273.15*units.K, color = "red", label = "CONTROL Temperature", linewidth = 1)
nlhtemp, = testskew.plot(eriethermo_nolake["Pressure"]*units.hPa, eriethermo_nolake["Temperature"]*units.K-273.15*units.K, color = "maroon", label = "NLH Temperature", linewidth = 1)
condew, = testskew.plot(eriethermo_control["Pressure"]*units.hPa, eriethermo_control["Dewpoint"]*units.K-273.15*units.K, color = "dodgerblue", label = "CONTROL Dewpoint", linewidth = 1)
nlhdew, = testskew.plot(eriethermo_nolake["Pressure"]*units.hPa, eriethermo_nolake["Dewpoint"]*units.K-273.15*units.K, color = "midnightblue", label = "NLH Dewpoint", linewidth = 1)
# prof = mcalc.parcel_profile(nalcmsmean["Pres"][10:]*units.hPa, nalcmsmean["Temp"][:10].mean(dim = "z")*units.K, nalcmsmean["Dewpoint"][:10].mean(dim = "z")*units.K)
# testskew.plot(nalcmsmean["Pres"][10:]*units.hPa, prof-273.15*units.K, linewidth = 2, color = "gold", label = "Mixed-Layer Parcel Path")
# testskew.plot_barbs(highdptmean["Pres"][:80:5]*units.hPa, highdptmean["UC"][:80:5]*units.meter/units.second, highdptmean["VC"][:80:5]*units.meter/units.second, color = "cyan", label = "Model Wind")
# # testskew.plot_barbs(highdptmean["Pres"][:80:5]*units.hPa, sound05_interpu[:80:5]*units.meter/units.second, sound05_interpv[:80:5]*units.meter/units.second, color = "white", label = "Obs Wind")
# # testskew.plot(soundds["PRES"]*units.hPa, sounddf["TEMP"]*units.K, color = "maroon")
# # testskew.plot(soundds["PRES"]*units.hPa, sounddf["DWPT"]*units.K, color = "darkgreen")
# # testskew.plot_barbs(soundds["pres"][::100], soundds["u_wind"][::100], soundds["v_wind"][::100])
dryads = testskew.plot_dry_adiabats(t0 = np.arange(-40, 200, 10)*units.degC, colors = "tomato", linewidths = 0.75, label = "Dry Adiabats", alpha = 0.5)
moistads = testskew.plot_moist_adiabats(t0 = np.arange(-40, 45, 5)*units.degC, colors = "dodgerblue", linewidths = 0.75, label = "Moist Adiabats", alpha = 0.5)
# testskew.plot_mixing_lines(pressure = np.arange(1025, 450, -25)*units.hPa, colors = "limegreen", linewidths = 0.75, label = ")
proflegend = testskew.ax.legend(framealpha = 0, fontsize = 8, loc = "upper right", bbox_to_anchor = (1.02, 1), handles = [contemp, nlhtemp, condew, nlhdew])
testskew.ax.add_artist(proflegend)
backlegend = testskew.ax.legend(framealpha = 0, fontsize = 8, loc = "lower right", handles = [dryads, moistads])
testskew.ax.add_artist(backlegend)
testskew.ax.set_ylabel("Pressure (hPa)", fontsize = 13)
testskew.ax.set_xlabel(r"Temperature ($\mathrm{\degree C}$)", fontsize = 13)
# testskew.ax.set_title("Lake Erie Thermodynamic Profiles at 1800 LT", fontsize = 13)
plt.savefig("../PaperFigs/skewtcomp.png")
plt.close()
del tfig; del testskew

In [ ]:
print(eriethermo_control["VaporMix"].sel(z = slice(0, 700)).mean())
print(eriethermo_nolake["VaporMix"].sel(z = slice(0, 700)).mean())
print(eriethermo_control["Dewpoint"].sel(z = slice(300, 700)).mean())
print(eriethermo_nolake["Dewpoint"].sel(z = slice(300, 700)).mean())

In [ ]:
print(eriethermo_control["Temperature"].sel(z = 700, method = "nearest").mean())
print(eriethermo_nolake["Temperature"].sel(z = 700, method = "nearest").mean())

## 4-Panel figure showing movement of convective bands

In [ ]:
ramscrs = ccrs.Stereographic(central_longitude = -80, central_latitude = 42)
plttimes = date_range("2010-01-02-234000", "2010-01-03-024000", freq = "H")
walt = 700
try:
    afiles
except:
    afiles = xr.open_mfdataset([f"{controlpath}/mvars-cart-{i.strftime('%Y-%m-%d-%H%M%S')}-g1.nc" for i in plttimes], preprocess = lambda x: x["w"], combine = "nested", concat_dim = "time")
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize = (5.5, 6), layout = "compressed", subplot_kw = {"projection": ramscrs}, dpi = 250)
axlabels = ["(a)", "(b)", "(c)", "(d)"]
for i, ax in enumerate(fig.get_axes()):
    ax = add_cartofeatures_wmovement(ax)
    left, width = 0, 0.1
    bottom, height = 0.9, 0.1
    right = left + width
    top = bottom + height
    p = plt.Rectangle((left, bottom), width, height, fill=True, zorder = 3, edgecolor = "black", linewidth = 0.2, facecolor = "white")
    p.set_transform(ax.transAxes)
    p.set_clip_on(False)
    ax.add_patch(p)
    ax.text(left+0.5*width, bottom+0.5*height, axlabels[i], fontsize = 9, transform = ax.transAxes, horizontalalignment = "center", verticalalignment = "center")
    ax.set_title(f"{plttimes[i].strftime('%d')} Jan - {plttimes[i].strftime('%H%M')} UTC")
    wmp = ax.pcolormesh(afiles.isel(time = 0)["x"].isel(x = slice(50, 650)), afiles.isel(time = 0)["y"].isel(y = slice(400, 1050)), afiles.isel(time = i).sel(z = walt, method = "nearest").isel(x = slice(50, 650), y = slice(400, 1050)), cmap = "RdBu_r", vmin = -3, vmax = 3, transform = ramscrs)
fig.suptitle(f"CONTROL Simulation {walt}m Vertical Velocity", fontsize = 12)
cbar = fig.colorbar(wmp, ax = [ax1, ax2, ax3, ax4], orientation = "horizontal", extend = "both", pad = 0.02, fraction = 0.05, aspect = 35)
cbar.set_label(label = "Vertical Velocity ($\mathrm{m \ s^{-1}}$)", fontsize = 10)
fig.savefig("../PaperFigs/wcomp_carto_4panel.png")
# plt.close()
del fig; del ax1; del ax2; del ax3; del ax4
afiles.close(); del afiles

## Figure showing locations of vertical cross-sections 1

In [ ]:
ix0s = [280, 310, 370, 385] #Southwest x points of the cross-sections
iy0s = [690, 600, 520, 500] #Southwest y points of the cross-sections
#These points were picked subjectively based on the 700 m vertical velocity field
angles = [30, 30, 50, 25] #Angle north of east of each cross-section
lengths = [100000, 100000, 100000, 100000] #Lengths, in m, of the cross-sections
xcdict = {}
xcnames = ["LH", "OP", "LE", "AP"] #Names of the cross sections. These mean "Lake Huron", "Ontario Peninsula", "Lake Erie", and "Allegheny Plateau", respectively
for i,_ in enumerate(ix0s):
    ix0 = ix0s[i]
    iy0 = iy0s[i]
    angle = angles[i]
    length = lengths[i]
    x0 = controlgrid["x"].isel(x = ix0); y0 = controlgrid["y"].isel(y = iy0); dxy = np.diff(controlgrid["x"])[0]
    darray = np.arange(0, length+1, dxy);  #Coordinates of the box in distance along the cross-section. Positive is to the northeast
    angle = np.pi/180*(angle) #Angle of the long axis of the box, with respect to the horizontal
    xbox = x0.values+np.cos(angle)*darray
    ybox = y0.values+np.sin(angle)*darray
    xcset = xr.DataArray(coords = {"y": ("distance", ybox), "x": ("distance", xbox), "distance": darray}, dims = ["distance"])
    xcdict[xcnames[i]] = xcset

In [ ]:
time2300z = datetime.strptime("2010-01-02-230000", "%Y-%m-%d-%H%M%S")
afile_control_onetime = xr.open_dataset(f"{controlpath}/mvars-cart-{time2300z.strftime('%Y-%m-%d-%H%M%S')}-g1.nc")
afile_nolake_onetime = xr.open_dataset(f"{nlhpath}/mvars-cart-{time2300z.strftime('%Y-%m-%d-%H%M%S')}-g1.nc")
moresnowpath = input("Enter the directory containing the ANALYSIS files for the MORESNOW simulation: ").rstrip("/ ")
if not os.path.exists(moresnowpath):
    raise Exception("MORESNOW directory not found!")
afile_moresnow_onetime = xr.open_dataset(f"{moresnowpath}/a-A-{time2300z.strftime('%Y-%m-%d-%H%M%S')}-g1.h5", engine = "h5netcdf", phony_dims = "sort")
# afile_control_onetime["SensibleHeatFlux"].isel(x = slice(400, 600), y = slice(650,850)).plot(cmap = "bwr")

In [ ]:
ramscrs = ccrs.Stereographic(central_longitude = -80, central_latitude = 42)
walt = 700
xclabels = {"LH": "1", "OP": "2", "LE": "3", "AP": "4"} #I realized that it's probably better to just number the cross-sections instead of using the 2-letter labeling
afile_control_onetime = xr.open_dataset(f"{controlpath}/mvars-cart-2010-01-02-230000-g1.nc")
afile_nolake_onetime = xr.open_dataset(f"{nlhpath}/mvars-cart-2010-01-02-230000-g1.nc")
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize = (5, 6), dpi = 200, layout = "constrained", subplot_kw = {"projection": ramscrs})
fig.get_layout_engine().set(wspace = 0, hspace = 0, w_pad = 4/72, h_pad = 4/72)
axlabels = ["(a)", "(b)"]
for i, ax in enumerate([ax1, ax2]):
    ax = add_cartofeatures_zoomdomain(ax)
    left, width = 0, 0.10
    bottom, height = 0.90, 0.1
    right = left + width
    top = bottom + height
    p = plt.Rectangle((left, bottom), width, height, fill=True, zorder = 3, edgecolor = "black", linewidth = 0.2, facecolor = "white")
    p.set_transform(ax.transAxes)
    p.set_clip_on(False)
    ax.add_patch(p)
    ax.text(left+0.5*width, bottom+0.5*height, axlabels[i], fontsize = 10, transform = ax.transAxes, horizontalalignment = "center", verticalalignment = "center")
    # tercmap = ax.pcolormesh(controlgrid["x"][50:650], controlgrid["y"][450:1050], controlgrid["Topo"][450:1050, 50:650], cmap = modtopocmap, vmin = 0, vmax = 1000, zorder = 0, transform = ramscrs)
for xcname in xcnames[:3]:
    ax1.plot([xcdict[xcname]["x"][0], xcdict[xcname]["x"][-1]], [xcdict[xcname]["y"][0], xcdict[xcname]["y"][-1]], linewidth = 0.5, color = "black", zorder = 2)
    ax1.annotate(xclabels[xcname], xy = (xcdict[xcname]["x"][-1]+5000, xcdict[xcname]["y"][-1]-8000), horizontalalignment = "left", color = "black", fontsize = 8)
walt = 700 #m AMSL, altitude at which to evalute w
# fig.suptitle(f"{walt} m AMSL Vertical Velocity at 02 Jan - {(time2300z-timedelta(hours = 5)).strftime('%H%M')} LT", fontsize = 14)
wmp = ax1.pcolormesh(controlgrid["x"].isel(x = slice(50, 650)), controlgrid["y"].isel(y = slice(400, 1050)), afile_control_onetime["w"].sel(z = walt, method = "nearest").isel(x = slice(50, 650), y = slice(400, 1050)), cmap = "RdBu_r", vmin = -3, vmax = 3, zorder = 1, transform = ramscrs)
ax2.pcolormesh(controlgrid["x"].isel(x = slice(50, 650)), controlgrid["y"].isel(y = slice(400, 1050)), afile_nolake_onetime["w"].sel(z = walt, method = "nearest").isel(x = slice(50, 650), y = slice(400, 1050)), cmap = "RdBu_r", vmin = -3, vmax = 3, zorder = 1, transform = ramscrs)
# (-80.75, -78.75, 42, 44.5)
ax2.plot([-80.75, -78.75], [42, 42], color = "black", linestyle = "--", linewidth = 0.5, transform = ccrs.PlateCarree(), zorder = 2) #Mark out the brown lake area
ax2.plot([-80.75, -78.75], [44.5, 44.5], color = "black", linestyle = "--", linewidth = 0.5, transform = ccrs.PlateCarree(), zorder = 2) #Mark out the brown lake area
ax2.plot([-80.75, -80.75], [42, 44.5], color = "black", linestyle = "--", linewidth = 0.5, transform = ccrs.PlateCarree(), zorder = 2) #Mark out the brown lake area
ax2.plot([-78.75, -78.75], [42, 44.5], color = "black", linestyle = "--", linewidth = 0.5, transform = ccrs.PlateCarree(), zorder = 2) #Mark out the brown lake area
axlabels2 = ["(c)", "(d)"]
for i, ax in enumerate([ax3, ax4]):
    ax = add_cartofeatures_brownlake(ax)
    # ax.set_aspect(0.6)
    left, width = 0, 0.15
    bottom, height = 0.90, 0.1
    right = left + width
    top = bottom + height
    p = plt.Rectangle((left, bottom), width, height, fill=True, zorder = 3, edgecolor = "black", linewidth = 0.2, facecolor = "white")
    p.set_transform(ax.transAxes)
    p.set_clip_on(False)
    ax.add_patch(p)
    ax.text(left+0.5*width, bottom+0.5*height, axlabels2[i], fontsize = 10, transform = ax.transAxes, horizontalalignment = "center", verticalalignment = "center")
ax3.pcolormesh(controlgrid["x"].isel(x = slice(430,610)), controlgrid["y"].isel(y = slice(550, 830)), afile_control_onetime["w"].sel(z = walt, method = "nearest").isel(x=slice(430,610), y = slice(550, 830)), vmin = -3, vmax = 3, cmap = "RdBu_r", zorder = 1, transform = ramscrs)
ax4.pcolormesh(controlgrid["x"].isel(x = slice(430,610)), controlgrid["y"].isel(y = slice(550, 830)), afile_nolake_onetime["w"].sel(z = walt, method = "nearest").isel(x=slice(430,610), y = slice(550, 830)), vmin = -3, vmax = 3, cmap = "RdBu_r", zorder = 1, transform = ramscrs)
# fig.suptitle(f"{walt} m AMSL Vertical Velocity at 02 Jan - 1800 LT", fontsize = 12)
ax1.set_title("CONTROL", fontsize = 12)
ax2.set_title("NLH", fontsize = 12)
# ax3.set_title("CONTROL", fontsize = 12)
# ax4.set_title("NLH", fontsize = 12)
cbar = fig.colorbar(wmp, ax = [ax1, ax2, ax3, ax4], orientation = "horizontal", fraction = 0.05, extend = "both", aspect = 30); cbar.set_label(f"{walt}m AMSL Vertical Velocity ($\mathrm{{m \ s^{{-1}}}}$)", fontsize = 12)
fig.savefig("../PaperFigs/wcomp_xcmark.png")
plt.close(); del fig; del ax1; del ax2;

In [ ]:
ramscrs = ccrs.Stereographic(central_longitude = -80, central_latitude = 42)
print("Plotting field Div")
ftime = time2300z.strftime("%Y-%m-%d-%H%M%S")
print(ftime)
afile_control = xr.open_dataset(f"{controlpath}/mvars-cart-{ftime}-g1.nc")
afile_nolake = xr.open_dataset(f"{nlhpath}/mvars-cart-{ftime}-g1.nc")
modtopocmap = truncate_colormap(mpl.colormaps.get_cmap("gist_earth"), minval = 0.5, maxval = 1, n = 128) #A truncated version of gist_earth to only show terrain height above sea level (gets rid of all the blue on the colorbar)
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize = (5, 5.5), dpi = 200, layout = "compressed", subplot_kw = {"projection": ramscrs})
axlabels = ["(a)", "(b)", "(c)", "(d)"]
fakecontour = Line2D([], [], color = "black", linewidth = 0.375, label = "Water Bodies")
for i, ax in enumerate(fig.get_axes()):
    ax = add_cartofeatures_zoomdomain(ax)
    left, width = 0, 0.1
    bottom, height = 0.90, 0.1
    right = left + width
    top = bottom + height
    p = plt.Rectangle((left, bottom), width, height, fill=True, zorder = 3, edgecolor = "black", linewidth = 0.2, facecolor = "white")
    p.set_transform(ax.transAxes)
    p.set_clip_on(False)
    ax.add_patch(p)
    ax.text(left+0.5*width, bottom+0.5*height, axlabels[i], fontsize = 10, transform = ax.transAxes, horizontalalignment = "center", verticalalignment = "center", zorder = 5, color = "black")
    tercmap = ax.pcolormesh(afile_control["x"][50:650], afile_control["y"][450:1050], afile_control["Topo"][450:1050, 50:650], cmap = modtopocmap, vmin = 0, vmax = 1000, zorder = 0, transform = ramscrs)
divalt1 = 350 #m AMSL, altitude at which to evaluate horizontal divergence on bottom row of plots.
divalt2 = 500 #m AMSL, altitude at which to evaluate horizontal divergence on top row of plot
fig.suptitle(f"Horizontal Divergence at {time2300z.strftime('%d')} Jan - {time2300z.strftime('%H%M')} UTC")
ax1.set_title(f"CONTROL {divalt1}m")
ax2.set_title(f"NLH {divalt1}m")
ax3.set_title(f"CONTROL {divalt2}m")
ax4.set_title(f"NLH {divalt2}m")
for xcname in xcnames[:3]:
    ax1.plot([xcdict[xcname]["x"][0], xcdict[xcname]["x"][-1]], [xcdict[xcname]["y"][0], xcdict[xcname]["y"][-1]], linewidth = 0.5, color = "black", zorder = 2)
    ax1.annotate(xclabels[xcname], xy = (xcdict[xcname]["x"][-1]+5000, xcdict[xcname]["y"][-1]-8000), horizontalalignment = "left", color = "black", fontsize = 8)
horizdiv_control_alt1 = np.gradient(afile_control["u"].sel(z = divalt1, method = "nearest").isel(x = slice(50, 650), y = slice(450, 1050)).values, 1000, axis = 1)+np.gradient(afile_control["v"].sel(z = divalt1, method = "nearest").isel(x = slice(50, 650), y = slice(450, 1050)).values, 1000, axis = 0)
horizdiv_control_alt2 = np.gradient(afile_control["u"].sel(z = divalt2, method = "nearest").isel(x = slice(50, 650), y = slice(450, 1050)).values, 1000, axis = 1)+np.gradient(afile_control["v"].sel(z = divalt2, method = "nearest").isel(x = slice(50, 650), y = slice(450, 1050)).values, 1000, axis = 0)
horizdiv_nolake_alt1 = np.gradient(afile_nolake["u"].sel(z = divalt1, method = "nearest").isel(x = slice(50, 650), y = slice(450, 1050)).values, 1000, axis = 1)+np.gradient(afile_nolake["v"].sel(z = divalt1, method = "nearest").isel(x = slice(50, 650), y = slice(450, 1050)).values, 1000, axis = 0)
horizdiv_nolake_alt2 = np.gradient(afile_nolake["u"].sel(z = divalt2, method = "nearest").isel(x = slice(50, 650), y = slice(450, 1050)).values, 1000, axis = 1)+np.gradient(afile_nolake["v"].sel(z = divalt2, method = "nearest").isel(x = slice(50, 650), y = slice(450, 1050)).values, 1000, axis = 0)
divmp = ax1.pcolormesh(afile_control["x"].isel(x = slice(50, 650)), afile_control["y"].isel(y = slice(450, 1050)), horizdiv_control_alt1, cmap = "BrBG", vmin = -5*10**(-3), vmax = 5*10**(-3), zorder = 1, transform = ramscrs)
ax2.pcolormesh(afile_control["x"].isel(x = slice(50, 650)), afile_control["y"].isel(y = slice(450, 1050)), horizdiv_nolake_alt1, cmap = "BrBG", vmin = -5*10**(-3), vmax = 5*10**(-3), zorder = 1, transform = ramscrs)
ax3.pcolormesh(afile_control["x"].isel(x = slice(50, 650)), afile_control["y"].isel(y = slice(450, 1050)), horizdiv_control_alt2, cmap = "BrBG", vmin = -5*10**(-3), vmax = 5*10**(-3), zorder = 1, transform = ramscrs)
ax4.pcolormesh(afile_control["x"].isel(x = slice(50, 650)), afile_control["y"].isel(y = slice(450, 1050)), horizdiv_nolake_alt2, cmap = "BrBG", vmin = -5*10**(-3), vmax = 5*10**(-3), zorder = 1, transform = ramscrs)
cbar1 = fig.colorbar(divmp, ax = [ax1, ax2, ax3, ax4], orientation = "horizontal", fraction = 0.05, aspect = 40, extend = "both", pad = 0.02); cbar1.set_label("Horizontal Divergence ($\mathrm{{s^{{-1}}}}$)", labelpad = 7)
cbar2 = fig.colorbar(tercmap, ax = [ax1, ax2, ax3, ax4], orientation = "vertical", location = "left", fraction = 0.05, aspect = 40, extend = "max", pad = 0.02); cbar2.set_label("Terrain height (m)", labelpad = 4)
cbar1.formatter.set_powerlimits((-1, 1))
fig.savefig(f"../PaperFigs/div_4panel_xcmark_{time2300z.strftime('%d-%H%M')}z.png")
plt.close();
del fig; del ax1; del ax2; del ax3; del ax4
del cbar1; del cbar2

In [ ]:
lhdata = afile_control_onetime[["PatchArea", "Topo", "HorizDiv", "Temperature", "LatentHeatFreezingTotal", "ThetaV", "LatentHeatCondensationTotal", "VaporMix", "u", "v", "w", "SensibleHeatFlux"]].interp(x = xcdict["LH"]["x"], y = xcdict["LH"]["y"])
lhtanwind = lhdata["u"]*np.cos(np.pi/180*angles[0])+lhdata["v"]*np.sin(np.pi/180*angles[0])
opdata = afile_control_onetime[["PatchArea", "Topo", "HorizDiv", "Temperature", "LatentHeatFreezingTotal", "ThetaV", "LatentHeatCondensationTotal", "VaporMix", "u", "v", "w", "SensibleHeatFlux"]].interp(x = xcdict["OP"]["x"], y = xcdict["OP"]["y"])
optanwind = opdata["u"]*np.cos(np.pi/180*angles[1])+opdata["v"]*np.sin(np.pi/180*angles[1])
ledata = afile_control_onetime[["PatchArea", "Topo", "HorizDiv", "Temperature", "LatentHeatFreezingTotal", "ThetaV", "LatentHeatCondensationTotal", "VaporMix", "u", "v", "w", "SensibleHeatFlux"]].interp(x = xcdict["LE"]["x"], y = xcdict["LE"]["y"])
letanwind = ledata["u"]*np.cos(np.pi/180*angles[2])+ledata["v"]*np.sin(np.pi/180*angles[2])
apdata = afile_control_onetime[["PatchArea", "Topo", "HorizDiv", "Temperature", "LatentHeatFreezingTotal", "ThetaV", "LatentHeatCondensationTotal", "VaporMix", "u", "v", "w", "SensibleHeatFlux"]].interp(x = xcdict["AP"]["x"], y = xcdict["AP"]["y"])
aptanwind = apdata["u"]*np.cos(np.pi/180*angles[3])+apdata["v"]*np.sin(np.pi/180*angles[3])

#Here, we calculate the wind speeds parallel to the direction of each cross-section, for use in the plots below

## Plot vertical cross-sections of vertical velocity and horizontal divergence

In [ ]:
ramscrs = ccrs.Stereographic(central_longitude = -80, central_latitude = 42)
fig, ((ax21, ax22), (ax31, ax32), (ax41, ax42)) = plt.subplots(3, 2, figsize = (6, 6.5), dpi = 200, layout = "constrained")
fig.get_layout_engine().set(hspace = 0, wspace = 0)
# ax11.remove(); ax12.remove()
# ax11 = fig.add_subplot(521, projection = ramscrs)
# ax12 = fig.add_subplot(522, projection = ramscrs)
axlabels = ["(a)", "(b)", "(c)", "(d)", "(e)", "(f)", "(g)", "(h)"]
# for ax in (ax11, ax12):
#     ax.set_aspect(1)
#     ax = add_cartofeatures_wmovement(ax)
#     for xcname in xcnames[:3]:
#         ax.plot([xcdict[xcname]["x"][0], xcdict[xcname]["x"][-1]], [xcdict[xcname]["y"][0], xcdict[xcname]["y"][-1]], linewidth = 0.5, color = "black", zorder = 2)
#         ax.annotate(xcname, xy = (xcdict[xcname]["x"][-1]+5000, xcdict[xcname]["y"][-1]-8000), horizontalalignment = "left", color = "black", fontsize = 8)
#     ax.plot([xcdict[xcnames[3]]["x"][0], xcdict[xcnames[3]]["x"][-1]], [xcdict[xcnames[3]]["y"][0], xcdict[xcnames[3]]["y"][-1]], linewidth = 0.5, color = "black", zorder = 2)
#     ax.annotate(xcnames[3], xy = (xcdict[xcnames[3]]["x"][0]-5000, xcdict[xcnames[3]]["y"][0]), horizontalalignment = "right", verticalalignment = "top", color = "black", fontsize = 8)
#     tercmap = ax.pcolormesh(controlgrid["x"][50:650], controlgrid["y"][400:1050], controlgrid["Topo"][400:1050, 50:650], cmap = modtopocmap, vmin = 0, vmax = 1000, zorder = 0, transform = ramscrs)
for i, ax in enumerate(fig.get_axes()):
    left, width = 0, 0.10
    bottom, height = 0.90, 0.1
    right = left + width
    top = bottom + height
    p = plt.Rectangle((left, bottom), width, height, fill=True, zorder = 3, edgecolor = "black", linewidth = 0.2, facecolor = "white")
    p.set_transform(ax.transAxes)
    p.set_clip_on(False)
    ax.add_patch(p)
    ax.text(left+0.5*width, bottom+0.5*height, axlabels[i], fontsize = 9, transform = ax.transAxes, horizontalalignment = "center", verticalalignment = "center")
# walt = 700 #m AMSL, altitude at which to evalute w
# divalt = 350 #m AMSL, altitude at which to evaluate horizontal divergence
# # fig.suptitle(f"Vertial Cross-Sections at 02 Jan - 1800 LT", fontsize = 12, y = 0.03)
# wmp = ax11.pcolormesh(controlgrid["x"].isel(x = slice(50, 650)), controlgrid["y"].isel(y = slice(400, 1050)), afile_control_onetime["w"].sel(z = walt, method = "nearest").isel(x = slice(50, 650), y = slice(400, 1050)), cmap = "RdBu_r", vmin = -3, vmax = 3, zorder = 1, transform = ramscrs)
# divmp = ax12.pcolormesh(controlgrid["x"].isel(x = slice(50, 650)), controlgrid["y"].isel(y = slice(400, 1050)), afile_control_onetime["HorizDiv"].sel(z = divalt, method = "nearest").isel(x = slice(50, 650), y = slice(400, 1050)), cmap = "BrBG", vmin = -5*10**(-3), vmax = 5*10**(-3), zorder = 1, transform = ramscrs)

# ax11.set_title("700m Vertical Velocity", fontsize = 12)
# ax12.set_title("350m Divergence", fontsize = 12)
# fig.suptitle("Vertical Cross-Sections at 1800 LT")
for ax in (ax21, ax22, ax31, ax32, ax41, ax42):
    if ax is not ax41:
        ax.tick_params(bottom = False, top = False, left = False, right = False, labelbottom = False, labeltop = False, labelleft = False, labelright = False)
    ax.set_aspect(30)
    ax.set_ylim(0, 2000)
ax41.tick_params(labelsize = 10)   
ax41.set_xticks(ticks = [0, 50000, 100000], labels = [0, 50, 100])
ax41.set_xlabel("Distance (km) ", fontsize = 10)
ax41.set_ylabel("Altitude (m)", fontsize = 10)
ax41.set_yticks(ticks = [0, 500, 1000, 1500])

ax21.set_title("CS 1 Vertical Velocity", fontsize = 12)
wmp = ax21.pcolormesh(lhdata["distance"], lhdata["z"].sel(z = slice(0, 2000)), lhdata["w"].sel(z = slice(0, 2000)), vmin = -3, vmax = 3, cmap = "RdBu_r")
qvmp = ax21.quiver(lhdata["distance"][1::3], lhdata["z"][0:60:4], convolve(lhtanwind[0:60,:], kernel = np.array([[1,2,3,2,1], [1,2,3,2,1], [1,2,3,2,1]]), normalize_kernel = True, nan_treatment = "interpolate", preserve_nan = True)[::4,1::3]-lhtanwind[0:60:4,1::3].mean(dim = "distance").values[:,None], lhdata["w"][0:60:4,1::3]*10, headwidth = 5, headlength = 5)
left, width = 0.68, 0.32
bottom, height = 0.85, 0.15
right = left + width
top = bottom + height
p = plt.Rectangle((left, bottom), width, height, fill=True, zorder = 1, edgecolor = "black", linewidth = 0.2, facecolor = "white")
p.set_transform(ax21.transAxes)
p.set_clip_on(False)
ax21.add_patch(p)
ax21.quiverkey(qvmp, U = 5, X = 0.73, Y = 0.92, label = r"5 $m \ s^{-1}$", labelpos = "E")
ax21.fill_between(lhdata["distance"], lhdata["Topo"], np.zeros(len(lhdata["distance"])), color = "tan")
ax21.fill_between(lhdata["distance"].where(lhdata["PatchArea"][0,:] > 0.99), lhdata["Topo"].where(lhdata["PatchArea"][0,:]>0.99), 1/2*lhdata["Topo"].where(lhdata["PatchArea"][0,:]>0.99), color = "blue")
ax22.set_title("CS 1 Divergence", fontsize = 12)
divmp = ax22.pcolormesh(lhdata["distance"], lhdata["z"].sel(z = slice(0, 2000)), lhdata["HorizDiv"].sel(z = slice(0, 2000)), vmin = -5*10**(-3), vmax = 5*10**(-3), cmap = "BrBG", zorder = 0)
ax22.fill_between(lhdata["distance"], lhdata["Topo"], np.zeros(len(lhdata["distance"])), color = "tan", zorder = 1)
ax22.fill_between(lhdata["distance"].where(lhdata["PatchArea"][0,:] > 0.99), lhdata["Topo"].where(lhdata["PatchArea"][0,:]>0.99), 1/2*lhdata["Topo"].where(lhdata["PatchArea"][0,:]>0.99), color = "blue")
ax22.contour(lhdata["distance"], lhdata["z"][0:60], 3*lhdata["LatentHeatCondensationTotal"][0:60,:]+3*lhdata["LatentHeatCondensationTotal"][0:60,:], levels = [2, 5, 10], colors = "red", linestyles = "--")
fakecontour = Line2D([], [], color = "red", linestyle = "--", linewidth = 1, label = r"Latent Heating Rate ($\mathrm{K \ hr^{-1}}$)")

ax31.set_title("CS 2 Vertical Velocity", fontsize = 12)
ax31.pcolormesh(opdata["distance"], opdata["z"].sel(z = slice(0, 2000)), opdata["w"].sel(z = slice(0, 2000)), vmin = -3, vmax = 3, cmap = "RdBu_r")
ax31.quiver(opdata["distance"][1::3], opdata["z"][0:60:4], convolve(optanwind[0:60,:], kernel = np.array([[1,2,3,2,1], [1,2,3,2,1], [1,2,3,2,1]]), normalize_kernel = True, nan_treatment = "interpolate", preserve_nan = True)[::4,1::3]-optanwind[0:60:4,1::3].mean(dim = "distance").values[:,None], opdata["w"][0:60:4,1::3]*10, headwidth = 5, headlength = 5)
ax31.fill_between(opdata["distance"], opdata["Topo"], np.zeros(len(opdata["distance"])), color = "tan")
ax31.fill_between(opdata["distance"].where(opdata["PatchArea"][0,:] > 0.99), opdata["Topo"].where(opdata["PatchArea"][0,:]>0.99), 1/2*opdata["Topo"].where(opdata["PatchArea"][0,:]>0.99), color = "blue")
ax32.set_title("CS 2 Divergence", fontsize = 12)
ax32.pcolormesh(opdata["distance"], opdata["z"].sel(z = slice(0, 2000)), opdata["HorizDiv"].sel(z = slice(0, 2000)), vmin = -5*10**(-3), vmax = 5*10**(-3), cmap = "BrBG", zorder = 0)
ax32.fill_between(opdata["distance"], opdata["Topo"], np.zeros(len(opdata["distance"])), color = "tan", zorder = 1)
ax32.fill_between(opdata["distance"].where(opdata["PatchArea"][0,:] > 0.99), opdata["Topo"].where(opdata["PatchArea"][0,:]>0.99), 1/2*opdata["Topo"].where(opdata["PatchArea"][0,:]>0.99), color = "blue")
ax32.contour(opdata["distance"], opdata["z"][0:60], 3*opdata["LatentHeatCondensationTotal"][0:60,:]+3*opdata["LatentHeatCondensationTotal"][0:60,:], levels = [2, 5, 10], colors = "red", linestyles = "--")

ax41.set_title("CS 3 Vertical Velocity", fontsize = 12)
ax41.pcolormesh(ledata["distance"], ledata["z"].sel(z = slice(0, 2000)), ledata["w"].sel(z = slice(0, 2000)), vmin = -3, vmax = 3, cmap = "RdBu_r")
ax41.quiver(ledata["distance"][1::3], ledata["z"][0:60:4], convolve(letanwind[0:60,:], kernel = np.array([[1,2,3,2,1], [1,2,3,2,1], [1,2,3,2,1]]), normalize_kernel = True, nan_treatment = "interpolate", preserve_nan = True)[::4,1::3]-letanwind[0:60:4,1::3].mean(dim = "distance").values[:,None], ledata["w"][0:60:4,1::3]*10, headwidth = 5, headlength = 5)
ax41.fill_between(ledata["distance"], ledata["Topo"], np.zeros(len(ledata["distance"])), color = "tan")
ax41.fill_between(ledata["distance"].where(ledata["PatchArea"][0,:] > 0.99), ledata["Topo"].where(ledata["PatchArea"][0,:]>0.99), 1/2*ledata["Topo"].where(ledata["PatchArea"][0,:]>0.99), color = "blue")
ax42.set_title("CS 3 Divergence", fontsize = 12)
ax42.pcolormesh(ledata["distance"], ledata["z"].sel(z = slice(0, 2000)), ledata["HorizDiv"].sel(z = slice(0, 2000)), vmin = -5*10**(-3), vmax = 5*10**(-3), cmap = "BrBG", zorder = 0)
ax42.fill_between(ledata["distance"], ledata["Topo"], np.zeros(len(ledata["distance"])), color = "tan", zorder = 1)
ax42.fill_between(ledata["distance"].where(ledata["PatchArea"][0,:] > 0.99), ledata["Topo"].where(ledata["PatchArea"][0,:]>0.99), 1/2*ledata["Topo"].where(ledata["PatchArea"][0,:]>0.99), color = "blue")
ax42.contour(ledata["distance"], ledata["z"][0:60], 3*ledata["LatentHeatCondensationTotal"][0:60,:]+3*ledata["LatentHeatCondensationTotal"][0:60,:], levels = [2, 5, 10], colors = "red", linestyles = "--")
ax42.legend(loc = "lower left", bbox_to_anchor = (-0.1, -0.45), fontsize = 10, framealpha = 0, handles = [fakecontour])

# ax51.set_title("CS 4 Vertical Velocity", fontsize = 12)
# ax51.pcolormesh(apdata["distance"], apdata["z"].sel(z = slice(0, 2000)), apdata["w"].sel(z = slice(0, 2000)), vmin = -3, vmax = 3, cmap = "RdBu_r")
# ax51.fill_between(apdata["distance"], apdata["Topo"], np.zeros(len(apdata["distance"])), color = "tan")
# ax51.fill_between(apdata["distance"].where(apdata["PatchArea"][0,:] > 0.99), apdata["Topo"].where(apdata["PatchArea"][0,:]>0.99), 1/2*apdata["Topo"].where(apdata["PatchArea"][0,:]>0.99), color = "blue")
# ax52.set_title("CS 4 Divergence", fontsize = 12)
# ax52.pcolormesh(apdata["distance"], apdata["z"].sel(z = slice(0, 2000)), apdata["HorizDiv"].sel(z = slice(0, 2000)), vmin = -5*10**(-3), vmax = 5*10**(-3), cmap = "BrBG", zorder = 0)
# ax52.fill_between(apdata["distance"], apdata["Topo"], np.zeros(len(apdata["distance"])), color = "tan", zorder = 1)
# ax52.fill_between(apdata["distance"].where(apdata["PatchArea"][0,:] > 0.99), apdata["Topo"].where(apdata["PatchArea"][0,:]>0.99), 1/2*apdata["Topo"].where(apdata["PatchArea"][0,:]>0.99), color = "blue")
# ax51.quiver(apdata["distance"][::2], apdata["z"][0:60:3], convolve(aptanwind[0:60], kernel = np.array([[1,2,3,2,1], [1,2,3,2,1], [1,2,3,2,1]]), normalize_kernel = True, nan_treatment = "interpolate", preserve_nan = True)[::3,::2], apdata["w"][0:60:3,::2]*10)
# ax52.contour(apdata["distance"], apdata["z"][0:65], 3*apdata["LatentHeatCondensationTotal"][0:65,:]+3*apdata["LatentHeatCondensationTotal"][0:65,:], levels = [2, 5, 10], colors = "red", linestyles = "--")
# ax52.legend(loc = "lower left", bbox_to_anchor = (-0.1, -0.45), fontsize = 10, framealpha = 0, handles = [fakecontour])
cbar1 = fig.colorbar(wmp, ax = [ax21, ax31, ax41], orientation = "horizontal", fraction = 0.05, extend = "both", pad = 0.01); cbar1.set_label(f"Vertical Velocity ($\mathrm{{m \ s^{{-1}}}}$)", fontsize = 10)
cbar2 = fig.colorbar(divmp, ax = [ax22, ax32, ax42], orientation = "horizontal", fraction = 0.05, extend = "both", pad = 0.01); cbar2.set_label(f"Horizontal Divergence ($\mathrm{{10^{{-3}} \ s ^{{-1}}}}$)", fontsize = 10)
cbar2.set_ticks([-5*10**(-3), -2.5*10**(-3), 0, 2.5*10**(-3), 5*10**(-3)], labels = [-5, -2.5, 0, 2.5, 5], fontsize = 10)
fig.savefig("../PaperFigs/crosssec_3row.png")
# plt.close()
del fig; del ax21; del ax22
del ax31; del ax32
del ax41; del ax42

## Plot surface sensible heat flux with arrow indicating snow-free land

### Sensible heat flux comparison between CONTROL, NLH, and MORESNOW

In [ ]:
afile_vartemp_onetime = xr.open_dataset(f"/moonbow/ascheb/les/2010/hires_vartemp/rams_output/a-A-{time2300z.strftime('%Y-%m-%d-%H%M%S')}-g1.h5", engine = "h5netcdf", phony_dims = "sort")

In [ ]:
ramscrs = ccrs.Stereographic(central_longitude = -80, central_latitude = 42)
walt = 700
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize = (7.5, 3.3), dpi = 200, layout = "compressed", subplot_kw = {"projection": ramscrs})
fig.get_layout_engine().set(wspace = 0, hspace = 0, w_pad = 4/72, h_pad = 4/72)
axlabels = ["(a)", "(b)", "(c)"]
for i, ax in enumerate([ax1, ax2, ax3]):
    ax = add_cartofeatures_zoomdomain(ax)
    left, width = 0, 0.10
    bottom, height = 0.90, 0.1
    right = left + width
    top = bottom + height
    p = plt.Rectangle((left, bottom), width, height, fill=True, zorder = 3, edgecolor = "black", linewidth = 0.2, facecolor = "white")
    p.set_transform(ax.transAxes)
    p.set_clip_on(False)
    ax.add_patch(p)
    ax.text(left+0.5*width, bottom+0.5*height, axlabels[i], fontsize = 10, transform = ax.transAxes, horizontalalignment = "center", verticalalignment = "center")

fig.suptitle(f"Surface Sensible Heat Flux at 02 Jan - {(time2300z).strftime('%H%M')} LT", fontsize = 12)
shmp = ax1.pcolormesh(controlgrid["x"].isel(x = slice(50, 650)), controlgrid["y"].isel(y = slice(450, 1050)), afile_control_onetime["SensibleHeatFlux"].isel(x = slice(50, 650), y = slice(450, 1050)), cmap = "bwr", vmin = -500, vmax = 500, zorder = 1, transform = ramscrs)
ax2.pcolormesh(controlgrid["x"].isel(x = slice(50, 650)), controlgrid["y"].isel(y = slice(450, 1050)), afile_nolake_onetime["SensibleHeatFlux"].isel(x = slice(50, 650), y = slice(450, 1050)), cmap = "bwr", vmin = -500, vmax = 500, zorder = 1, transform = ramscrs)
ax3.pcolormesh(controlgrid["x"].isel(x = slice(50, 650)), controlgrid["y"].isel(y = slice(450, 1050)), 1004*afile_moresnow_onetime["SFLUX_T"][451:1051, 51:651].values, cmap = "bwr", vmin = -500, vmax = 500, zorder = 1, transform = ramscrs)
cbar = fig.colorbar(shmp, ax = [ax1,ax2,ax3], orientation = "horizontal", fraction = 0.05, extend = "both", aspect = 25); cbar.set_label(f"Sensible Heat Flux ($\mathrm{{W \ m^{{-2}}}}$)", fontsize = 10)
ax1.annotate("", xy = (-79.9, 43.6), xytext = (-78.5, 44.5), textcoords = "data", horizontalalignment = "right", arrowprops = dict(facecolor = "black", shrink = 0.1, width = 1, headwidth = 4, headlength = 5), transform = ccrs.PlateCarree())
ax2.annotate("", xy = (-79.9, 43.6), xytext = (-78.5, 44.5), textcoords = "data", horizontalalignment = "right", arrowprops = dict(facecolor = "black", shrink = 0.1, width = 1, headwidth = 4, headlength = 5), transform = ccrs.PlateCarree())
ax3.annotate("", xy = (-79.9, 43.6), xytext = (-78.5, 44.5), textcoords = "data", horizontalalignment = "right", arrowprops = dict(facecolor = "black", shrink = 0.1, width = 1, headwidth = 4, headlength = 5), transform = ccrs.PlateCarree())
ax1.set_title("CONTROL", fontsize = 12)
ax2.set_title("NLH", fontsize = 12)
ax3.set_title("MORESNOW")
fig.savefig(f"../PaperFigs/shcomp_carto_{time2300z.strftime('%d-%H%M')}z_arrow.png")
# plt.close()
del fig; del ax1; del ax2; del ax3; del cbar

## Plot ERA5 and GLSEA Data

In [ ]:
eraair =xr.open_dataset("era5_jan2010_air.grib", engine = "cfgrib")

In [ ]:
eraground = xr.open_dataset("era5_jan2010_ground.grib", engine = "cfgrib")
# import netcdf4
laketemps = xr.open_dataset("2010_002_glsea_sst.nc", engine = "netcdf4", decode_cf = False)

In [ ]:
from matplotlib.colors import LinearSegmentedColormap
gcmap = LinearSegmentedColormap.from_list(name = "GreyMap", colors = [[1,1,1,1], [0.5, 0.5, 0.5, 1]], N=256)

mstokts = 1.944 #1 m/s is 1.944 kts - use in wind barb plots
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize = (4.1, 6), dpi = 300, layout = "compressed", subplot_kw = {"projection": ccrs.LambertConformal(central_longitude = -100, central_latitude = 50)})
eratime = np.datetime64("2010-01-03T00:00:00")
ax1.set_extent((-130, -70, 25, 70), crs = ccrs.PlateCarree())
ax1.set_aspect(1.3)
ax2.set_extent((-92.5, -76.5, 41.75, 47.75), crs = ccrs.PlateCarree())
ax2.set_aspect(1.15)

from matplotlib import ticker as mticker
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
for axsub in (ax3, ax4):
    axsub.set_extent((-95, -75, 38, 55), crs = ccrs.PlateCarree())
    # ax.add_feature(cfeature.STATES_PROVINCES, color = "white", zorder = 2)
axlabels = ["(a)", "(b)", "(c)", "(d)"]
for i, ax in enumerate([ax1, ax2, ax3, ax4]):
    left, width = 0, 0.1
    bottom, height = 0.9, 0.1
    right = left + width
    top = bottom + height
    p = plt.Rectangle((left, bottom), width, height, fill=True, zorder = 3, edgecolor = "black", linewidth = 0.3, facecolor = "white")
    p.set_transform(ax.transAxes)
    p.set_clip_on(False)
    ax.add_patch(p)
    ax.text(left+0.5*width, bottom+0.5*height, axlabels[i], fontsize = 8, transform = ax.transAxes, horizontalalignment = "center", verticalalignment = "center")
    
ax1.add_feature(cfeature.NaturalEarthFeature('physical', 'coastline', '50m', edgecolor = 'darkgrey', facecolor = "none", linewidth = 0.3, zorder = 2))
ax1.add_feature(cfeature.NaturalEarthFeature('cultural', 'admin_0_boundary_lines_land', '50m', edgecolor = "black", facecolor = "none", linewidth = 0.3, zorder = 2))
# ax.add_feature(cfeature.NaturalEarthFeature('physical', 'lakes', '50m',
#                                             edgecolor='navy',
#                                             facecolor="none", linewidth = 0.2), zorder = 1)
# ax.add_feature(cfeature.NaturalEarthFeature(category='cultural', name='admin_1_states_provinces_lines', facecolor='none', scale = "50m", edgecolor='darkgrey', zorder = 2))
ax1.add_feature(cfeature.NaturalEarthFeature(category="cultural", name = "admin_1_states_provinces_lakes", facecolor = "none", scale = "50m", edgecolor = "black", zorder = 2, linewidth = 0.3))

ax2.add_feature(cfeature.NaturalEarthFeature('physical', 'coastline', '50m', edgecolor = 'black', facecolor = "none", linewidth = 0.3, zorder = 2))
# ax2.add_feature(cfeature.NaturalEarthFeature('cultural', 'admin_0_boundary_lines_land', '50m', edgecolor = "black", facecolor = "none", linewidth = 0.3, zorder = 2))
    # ax.add_feature(cfeature.NaturalEarthFeature('physical', 'lakes', '50m',
    #                                             edgecolor='navy',
    #                                             facecolor="none", linewidth = 0.2), zorder = 1)
    # ax.add_feature(cfeature.NaturalEarthFeature(category='cultural', name='admin_1_states_provinces_lines', facecolor='none', scale = "50m", edgecolor='darkgrey', zorder = 2))
ax2.add_feature(cfeature.NaturalEarthFeature(category="cultural", name = "admin_1_states_provinces_lakes", facecolor = "none", scale = "50m", edgecolor = "black", zorder = 2, linewidth = 0.3))
    
for ax in (ax3, ax4):
    ax.add_feature(cfeature.NaturalEarthFeature('physical', 'coastline', '50m', edgecolor = 'black', facecolor = "none", linewidth = 0.3, zorder = 2))
    ax.add_feature(cfeature.NaturalEarthFeature('cultural', 'admin_0_boundary_lines_land', '50m', edgecolor = "black", facecolor = "none", linewidth = 0.3, zorder = 2))
    # ax.add_feature(cfeature.NaturalEarthFeature('physical', 'lakes', '50m',
    #                                             edgecolor='navy',
    #                                             facecolor="none", linewidth = 0.2), zorder = 1)
    # ax.add_feature(cfeature.NaturalEarthFeature(category='cultural', name='admin_1_states_provinces_lines', facecolor='none', scale = "50m", edgecolor='darkgrey', zorder = 2))
    ax.add_feature(cfeature.NaturalEarthFeature(category="cultural", name = "admin_1_states_provinces_lakes", facecolor = "none", scale = "50m", edgecolor = "black", zorder = 2, linewidth = 0.3))
    # ax.yaxis.tick_right()
    # ax.set_xticks([-95, -85, -75], crs=ccrs.PlateCarree())
    # ax.set_yticks([40, 45, 50, 55], crs=ccrs.PlateCarree())
    # lon_formatter = LongitudeFormatter(zero_direction_label=True)
    # lat_formatter = LatitudeFormatter()
    # ax.xaxis.set_major_formatter(lon_formatter)
    # ax.yaxis.set_major_formatter(lat_formatter)
    gl34 = ax.gridlines(crs = ccrs.PlateCarree(), draw_labels = {"bottom": "x", "left": "y"}, x_inline = False, y_inline = False, dms = True)
    gl34.xlocator = mticker.FixedLocator([-90, -85, -80])
    gl34.ylocator = mticker.FixedLocator([40, 45, 50, 55])
    gl34.bottom_labels = True
    gl34.left_labels   = True
    gl34.xlines = False
    gl34.ylines = False
    gl34.top_labels    = False
    gl34.right_labels  = False
    gl34.xlabel_style = {'size': 8, 'color': 'black', 'rotation': 0, "horizontalalignment": "right"}
    gl34.ylabel_style = {'size': 8, 'color': 'black', 'rotation': 0, "horizontalalignment": "right"}
    
gl1 = ax1.gridlines(crs = ccrs.PlateCarree(), draw_labels = {"bottom": "x", "left": "y"}, x_inline = False, y_inline = False)
gl1.xlocator = mticker.FixedLocator([-120, -100, -80])
gl1.ylocator = mticker.FixedLocator([25, 35, 45, 55])
gl1.bottom_labels = True
gl1.left_labels   = True
gl1.xlines = False
gl1.ylines = False
gl1.top_labels    = False
gl1.right_labels  = False
gl1.xlabel_style = {'size': 8, 'color': 'black', 'rotation': 0, "horizontalalignment": "center", 'rotation_mode': 'default'}
gl1.ylabel_style = {'size': 8, 'color': 'black', 'rotation': 0, "horizontalalignment": "center", 'rotation_mode': 'default'}

gl2 = ax2.gridlines(crs = ccrs.PlateCarree(), draw_labels = {"bottom": "x", "left": "y"}, x_inline = False, y_inline = False)
gl2.xlocator = mticker.FixedLocator([-90, -80])
gl2.ylocator = mticker.FixedLocator([43, 46])
gl2.bottom_labels = True
gl2.left_labels   = True
gl2.xlines = False
gl2.ylines = False
gl2.top_labels = False
gl2.right_labels = False
gl2.xlabel_style = {'size': 8, 'color': 'black', 'rotation': 0, "horizontalalignment": "right", 'rotation_mode': 'default'}
gl2.ylabel_style = {'size': 8, 'color': 'black', 'rotation': 0, "horizontalalignment": "right", 'rotation_mode': 'default'}

# fig.suptitle(f"Synoptic Conditions on {eratime.astype(datetime).strftime('%d')} Jan at {eratime.astype(datetime).strftime('%H%M')} UTC", fontsize = 14)
ax1.set_title(f"500mb Heights \n and Winds", fontsize = 9)
heightmp = ax1.contour(eraair["longitude"].sel(longitude = slice(-150, -65)), eraair["latitude"].sel(latitude = slice(75, 15)), 1/9.81*eraair["z"].sel(time = eratime, isobaricInhPa = 500, longitude = slice(-150, -65), latitude = slice(75, 15)), levels = np.arange(5220, 5820, 60), colors = "black", linestyles = "--", transform = ccrs.PlateCarree(), zorder = 0)
ax1.clabel(heightmp, inline = True, inline_spacing = 5, fontsize = 6, zorder = 2)
ax1.barbs(eraair["longitude"][::25], eraair["latitude"][::25], mstokts*eraair["u"].sel(time = eratime, isobaricInhPa = 500)[::25, ::25].values, mstokts*eraair["v"].sel(time = eratime, isobaricInhPa = 500)[::25,::25].values, color = "dodgerblue", transform = ccrs.PlateCarree(), length = 4, zorder = 1, linewidth = 0.3)
ax3.set_title(f"850mb Temperature \n and Winds", fontsize = 9)
ax3.barbs(eraair["longitude"][::8], eraair["latitude"][::8], mstokts*eraair["u"].sel(time = eratime, isobaricInhPa = 850).where(1/100*eraground["sp"].sel(time = eratime)>850)[::8, ::8].values, mstokts*eraair["v"].sel(time = eratime, isobaricInhPa = 850).where(1/100*eraground["sp"].sel(time = eratime)>850)[::8, ::8].values, color = "black", transform = ccrs.PlateCarree(), length = 4, zorder = 1, linewidth = 0.3)
ax3.pcolormesh(eraair["longitude"], eraair["latitude"], eraair["t"].sel(time = eratime, isobaricInhPa = 850).where(1/100*eraground["sp"].sel(time = eratime)>850)-273.15, cmap = "Spectral_r", transform = ccrs.PlateCarree(), zorder = 0, vmin = -28, vmax = 2)
lonslice = slice(-100, -67)
latslice = slice(57, 35)
ax4.set_title(f"Surface Temperature, \n Pressure, and Winds", fontsize = 9)
stempmp = ax4.pcolormesh(eraground["longitude"].sel(longitude = lonslice), eraground["latitude"].sel(latitude = latslice), eraground["t2m"].sel(time = eratime, longitude = lonslice, latitude = latslice)-273.15, cmap = "Spectral_r", transform = ccrs.PlateCarree(), zorder = 0, vmin = -28, vmax = 2)
mslpmp = ax4.contour(eraground["longitude"].sel(longitude = lonslice), eraground["latitude"].sel(latitude = latslice), 1/100*eraground["msl"].sel(time = eratime, longitude = lonslice, latitude = latslice), levels = [980, 984, 988, 992, 996, 1000, 1004, 1008, 1012, 1016, 1020, 1024, 1028, 1032, 1036, 1040], colors = "black", linestyles = "--", transform = ccrs.PlateCarree(), zorder = 1)
ax4.clabel(mslpmp, inline = True, fontsize = 6, levels = [1008, 1016, 1024, 1032], zorder = 2)
ax4.barbs(eraair["longitude"].sel(longitude = lonslice)[::8], eraair["latitude"].sel(latitude = latslice)[::8], mstokts*eraground["u10"].sel(time = eratime, longitude = lonslice, latitude = latslice)[::8,::8].values, mstokts*eraground["v10"].sel(time = eratime, longitude = lonslice, latitude = latslice)[::8,::8].values, color = "black", transform = ccrs.PlateCarree(), length = 4, zorder = 1, linewidth = 0.3)
ax2.set_title(f"Lake Surface \n Temperature", fontsize = 9)
laketmp = ax2.pcolormesh(laketemps["lon"], laketemps["lat"], laketemps["sst"].isel(time=0).where(laketemps["sst"].isel(time = 0)>0.2), vmin = 0, vmax = 6, cmap = "jet", transform = ccrs.PlateCarree(), zorder = 0)
#GLSEA assigns a water temperature of 0.2 C to areas of nonzero ice cover
ax2.pcolormesh(laketemps["lon"], laketemps["lat"], laketemps["sst"].isel(time=0).where(laketemps["sst"].isel(time = 0)==0.2), cmap = gcmap, vmin = 0, vmax = 0.2, transform = ccrs.PlateCarree(), zorder = 1)
ax2.set_position([0.6, 0.648, 0.37, 0.25])
aircbar = fig.colorbar(stempmp, ax = [ax1, ax3, ax4], orientation = "horizontal", extend = "both", fraction = 0.05, aspect = 35); aircbar.set_label(label = r"Air Temperature ($\degree C$)", fontsize = 8)
aircbar.ax.tick_params(labelsize = 8)
lkcbar = fig.colorbar(laketmp, ax = [ax2], orientation = "horizontal", extend = "max", fraction = 0.05, pad = 0.1);
lkcbar.set_label(label = r"Water Temperature ($\degree C$)", fontsize = 8)
lkcbar.ax.tick_params(labelsize = 8)
fig.savefig("../PaperFigs/synoptic.png")
plt.close(); del fig; del ax1; del ax2; del ax3; del ax4